#Stacking energies

In [2]:
#Loading dependencies
from Bio.PDB import Vector, PDBIO
from Bio.PDB.Residue import Residue
from Bio.PDB.StructureBuilder import StructureBuilder
from Bio.PDB.vectors import calc_angle
import Bio.PDB.Atom as Atom

from math import sqrt, pow, pi, cos, sin, pi, exp, log
from bisect import bisect

from numpy import sqrt, sum, dot, transpose, eye, array, matrix, zeros, log, exp, pi, power, transpose
from random import choice, random

import pickle

from numpy import *
from numpy.random import random, uniform
from numpy.linalg import inv, det

import random
from random import randint
import numpy.random
from numpy.random import random, uniform
from random import vonmisesvariate



import numpy as np
import pandas as pd
import os
import pickle
from tqdm import tqdm
import re
from scipy.spatial.distance import cdist
from itertools import combinations, product


FloatType = float
IntType = int
NoneType = None

In [3]:
path = '/Users/eric1999/Desktop/Eric/'
os. chdir(path)

pathML = '/Users/eric1999/Desktop/ML-stacking-noh_Eric/caca/'
pathPDB = '/Users/eric1999/Desktop/Eric/'
pdbfile = 'structure_999.pdb'
ofile = 'out.csv'

In [4]:
## Define functions
def potency2(X):
#    X3 = [ 1/(x**3) for x in X ]
    X6 = [ 1/(x**6) for x in X ]
#    X9 = [ 1/(x**9) for x in X ]
#    X10 = [ 1/(x**10) for x in X ]
    X12 = [ 1/(x**12) for x in X ]
    X = [ 1/x for x in X ]
#    out = np.array([X + X3 + X6 + X9 + X10 + X12])
    out = np.array([X + X6 + X12])
    return out

def parsexyz(pdbfile, path):
    x = {}
    y = {}
    z = {}
    count = 0
    for line in open(os.path.join(os.path.abspath(path), pdbfile)):
        splited = line.split()
        id = splited[0]
        if id == "ATOM":
            x[count] = float(splited[6])
            y[count] = float(splited[7])
            z[count] = float(splited[8])
            count = count + 1
    #Convert into dataframe
    df = {'x':x, 'y':y, 'z':z}
    df = pd.DataFrame(data=df)
    return df

def parsepdb(pdbfile, path):
    eleno = {}
    elety = {}
    resid = {}
    resno = {}
    x = {}
    y = {}
    z = {}
    count = 0
    for line in open(os.path.join(os.path.abspath(path), pdbfile)):
        splited = line.split()
        id = splited[0]
        if id == "ATOM":
            eleno[count] = int(splited[1])
            elety[count] = splited[2]
            resid[count] = splited[3]
            resno[count] = int(splited[5])
            x[count] = float(splited[6])
            y[count] = float(splited[7])
            z[count] = float(splited[8])
            count = count + 1
    #Convert into dataframe
    df = {'eleno': eleno, 'elety': elety, 'resid': resid, 'resno':resno, 'x':x, 'y':y, 'z':z}
    df = pd.DataFrame(data=df)
    df['resid'] = df['resid'].replace("5", "", regex=True)
    df['resid'] = df['resid'].replace("3", "", regex=True)
    return df

def topology(pdbfile, path):
    df = parsepdb(pdbfile=pdbfile, path=path)
    ## Get data for C1' atom of all residues
    mindf = df[df.elety == "C4"]
    ## Get combinations of all against all atoms
    combs_close = list(combinations(mindf.resno, 2))
    drop = df['elety'].isin(to_drop)
    df2 = df[~drop]
    df2.index = range(0, len(df2.index))
    sorterA=["C1'", "N9", "C8", "N7", "C5", "C6", "N6","N1", "C2", "N3", "C4"]
    sorterC=["C1'", "N1", "C6", "C5", "C4", "N4","N3", "C2", "O2"]
    sorterG=["C1'", "N9", "C8", "N7", "C5", "C6", "O6", "N1","C2", "N2","N3", "C4"]
    sorterU=["C1'", "N1", "C6","C5","C4", "O4", "N3","C2", "O2"]
    sorterIndexA = dict(zip(sorterA,range(len(sorterA))))
    sorterIndexC = dict(zip(sorterC,range(len(sorterC))))
    sorterIndexG = dict(zip(sorterG,range(len(sorterG))))
    sorterIndexU = dict(zip(sorterU,range(len(sorterU))))
    pairs=[]
    ix=[]
    combs=[]
    for i in combs_close:
        ## Get residue names of the pair
        resid1 = mindf.loc[mindf['resno'] == i[0], ['resid']].to_numpy()
        resid2 = mindf.loc[mindf['resno'] == i[1], ['resid']].to_numpy()
        pair = str(resid1 + resid2)[3:5]
        ## If not recognized, swap it
        if not pair in listofpairs:
            i = (i[1], i[0])
            resid1 = mindf.loc[mindf['resno'] == i[0], ['resid']].to_numpy()
            resid2 = mindf.loc[mindf['resno'] == i[1], ['resid']].to_numpy()
            pair = str(resid1 + resid2)[3:5]
        ## Save pair
        pairs.append(pair)
        ## Get indices for the first base according with pair and desired order of atoms
        if resid1 == 'A':
            sorterIndex = sorterIndexA
        elif resid1 == 'C':
            sorterIndex = sorterIndexC
        elif resid1 == 'G':
            sorterIndex = sorterIndexG
        elif resid1 == 'U':
            sorterIndex = sorterIndexU
        #data_base_1=df2[df2.resno == i[0]]
        #ix1=data_base_1.iloc[data_base_1['elety'].map(sorterIndex),].index
        data_base_1=df2[df2.resno == i[0]]
        data_base_1.insert(len(data_base_1.columns), 'rank', data_base_1['elety'].map(sorterIndex))
        ix1=data_base_1.sort_values(['rank']).index
        ## Get indices for the second base according with pair and desired order of atoms
        if resid2 == 'A':
            sorterIndex = sorterIndexA
        elif resid2 == 'C':
            sorterIndex = sorterIndexC
        elif resid2 == 'G':
            sorterIndex = sorterIndexG
        elif resid2 == 'U':
            sorterIndex = sorterIndexU
        data_base_2=df2[df2.resno == i[1]]
        data_base_2.insert(len(data_base_2.columns), 'rank', data_base_2['elety'].map(sorterIndex))
        ix2=data_base_2.sort_values(['rank']).index
        #ix1=data_base_1.index[data_base_1.resno == i[0]]
        #ix2=data_base_2.index[data_base_2.resno == i[1]]
        ix.append([ix1, ix2])
        combs.append(list(product(ix1, ix2)))
    return [combs_close, pairs, ix, combs, drop]

def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

## Define additional dependencies
to_drop = ["P", "OP1", "OP2", "OP3", "HO5'", "O5'", "C5'", "H5'", "H5''", "C4'", "H4'", "O4'", "H1'", "C3'", 
           "H3'", "C2'", "H2'", "O2'", "HO2'", "O3'", "HO3'", "H11'", "H12'", "H13'",
          "H8", "H61", "H62", "H2", "H6",  "H5", "H41", "H42",  "H1","H21","H22", "H3"]
to_keep = ["N1", "C2", "N3", "C4", "C5", "C6"]
listofpairs = ['AA', 'AC', 'AG', 'AU', 'CC', 'CU', 'GC', 'GG', 'GU', 'UU']

## Define model filenames
#AA_modelfile = pathML + '/AA_model_tpot.sav'
#AC_modelfile = pathML + '/AC_model_tpot.sav'
#AG_modelfile = pathML + '/AG_model_tpot.sav'
#AU_modelfile = pathML + '/AU_model_tpot.sav'
#CC_modelfile = pathML + '/CC_model_tpot.sav'
#CU_modelfile = pathML + '/CU_model_tpot.sav'
#GC_modelfile = pathML + '/GC_model_tpot.sav'
#GG_modelfile = pathML + '/GG_model_tpot.sav'
#GU_modelfile = pathML + '/GU_model_tpot.sav'
#UU_modelfile = pathML + '/UU_model_tpot.sav'
AA_modelfile = pathML + '/AA/model_scanned.sav'
AC_modelfile = pathML + '/AC/model_scanned.sav'
AG_modelfile = pathML + '/AG/model_scanned.sav'
AU_modelfile = pathML + '/AU/model_scanned.sav'
CC_modelfile = pathML + '/CC/model_scanned.sav'
CU_modelfile = pathML + '/CU/model_scanned.sav'
GC_modelfile = pathML + '/GC/model_scanned.sav'
GG_modelfile = pathML + '/GG/model_scanned.sav'
GU_modelfile = pathML + '/GU/model_scanned.sav'
UU_modelfile = pathML + '/UU/model_scanned.sav'

# Read models from disk
AA_model = pickle.load(open(AA_modelfile, 'rb'))
AC_model = pickle.load(open(AC_modelfile, 'rb'))
AG_model = pickle.load(open(AG_modelfile, 'rb'))
AU_model = pickle.load(open(AU_modelfile, 'rb'))
CC_model = pickle.load(open(CC_modelfile, 'rb'))
CU_model = pickle.load(open(CU_modelfile, 'rb'))
GC_model = pickle.load(open(GC_modelfile, 'rb'))
GG_model = pickle.load(open(GG_modelfile, 'rb'))
GU_model = pickle.load(open(GU_modelfile, 'rb'))
UU_model = pickle.load(open(UU_modelfile, 'rb'))

#Load and Run BARNACLE

In [5]:
StringType = str

to_one_letter_code = {'  A':'A', '  C':'C', '  G':'G', '  T':'T','  U':'U'}
to_three_letter_code = {'A':'  A', 'C':'  C', 'G':'  G', 'T':'  T','U':'  U'}
purine_one_letter_codes = ['A','G']
pyrimidine_one_letter_codes = ['C','T','U']
rna_one_letter_codes = ['A','C','G','U']
rna_three_letter_codes = ['  A','  C','  G','  U']

In [6]:
#################################################
# Parameters for the 20 Von Mises distributions #
#################################################
# The mean parameters (mu)
angle_mus = \
array([[ 3.54495134],
       [ 2.7128456 ],
       [ 1.67397508],
       [ 3.67269534],
       [ 5.00398034],
       [ 4.33719128],
       [ 5.1750921 ],
       [ 5.03051162],
       [ 1.42501808],
       [ 3.17184663],
       [ 0.99034376],
       [ 0.92521764],
       [ 2.57873053],
       [ 5.14146274],
       [ 3.45323421],
       [ 3.25118223],
       [ 3.59081053],
       [ 3.940963  ],
       [ 3.02828096],
       [ 1.48844185]])

# The concentration parameters (kappa)
angle_kappas = \
array([   1.18420695,    1.71027755,    4.17125563,   59.24947583,
         13.92277433,    5.5766314 ,   65.35858804,   72.74673231,
        338.52216605,    4.14872652,   20.5386044 ,  118.35753241,
         22.9723558 ,    6.02527586,   78.16547085,   44.19262618,
          6.95023602,   10.21467289,   49.74238856,   48.41417442])

#################################################
# Transition probabilities for the hidden state #
#################################################

# Distribution for hidden states in the first slice
#
# The probability for hidden value 'hd' given angle id 'aid' is given by:
#
#    hd0_cpd[aid, hd]

hd0_cpd = \
array([[  1.83520599e-01,   1.00000000e-50,   9.36329588e-02,
          9.73782772e-02,   1.00000000e-50,   1.00000000e-50,
          1.19850187e-01,   1.00000000e-50,   2.99625468e-02,
          1.00000000e-50,   7.11610487e-02,   7.49063670e-02,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          2.88389513e-01,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   4.11985019e-02],
       [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50],
       [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50],
       [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50],
       [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50],
       [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50],
       [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50]])

# Distribution for hidden states in the remaining slices
#
# The probability for hidden value 'hd' given the angle id 'aid' and the
# previous hidden value 'prev_hd' is given by:
#
#    hd0_cpd[aid, prev_hd, hd]

hd1_cpd = \
array([[[  8.66666667e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.33333333e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  9.56521739e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   2.17391304e-03,   1.00000000e-50,
           5.86956522e-01,   1.00000000e-50,   1.00000000e-50,
           1.84782609e-01,   1.00000000e-50,   1.00000000e-50,
           3.69565217e-02,   3.04347826e-02,   6.30434783e-02,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           2.86407767e-01,   1.00000000e-50,   1.00000000e-50,
           2.42718447e-01,   9.22330097e-02,   1.00000000e-50,
           1.00000000e-50,   1.40776699e-01,   1.00000000e-50,
           2.37864078e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   3.15789474e-01,   1.00000000e-50,
           6.84210526e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.25000000e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   8.75000000e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   2.18181818e-01,
           1.00000000e-50,   5.27272727e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.63636364e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   9.09090909e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.90476190e-01,   1.00000000e-50,   1.00000000e-50,
           4.44444444e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   3.65079365e-01,
           1.00000000e-50,   1.00000000e-50]],

       [[  1.00000000e-50,   1.00000000e-50,   3.06451613e-01,
           1.00000000e-50,   2.94930876e-01,   2.76497696e-02,
           3.91705069e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   6.22119816e-02,
           1.00000000e-50,   2.41935484e-01,   1.00000000e-50,
           1.84331797e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   9.21658986e-03],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           2.80487805e-01,   1.00000000e-50,   1.00000000e-50,
           4.87804878e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           7.31707317e-02,   1.00000000e-50,   1.00000000e-50,
           2.43902439e-02,   4.39024390e-01,   1.34146341e-01,
           1.00000000e-50,   1.00000000e-50],
        [  1.76000000e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           4.24000000e-01,   2.80000000e-01,   1.00000000e-50,
           1.20000000e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           7.01754386e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.57894737e-01,   7.71929825e-01],
        [  1.00000000e-50,   1.00000000e-50,   2.85714286e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   7.14285714e-01,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.81818182e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           4.54545455e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   7.72727273e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   8.75000000e-02,   1.00000000e-50,
           6.25000000e-03,   5.00000000e-02,   1.12500000e-01,
           1.00000000e-50,   1.37500000e-01,   1.00000000e-50,
           6.06250000e-01,   1.00000000e-50]],

       [[  5.05084746e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   5.08474576e-02,   2.71186441e-02,
           1.00000000e-50,   1.00000000e-50,   6.77966102e-02,
           1.00000000e-50,   2.71186441e-01,   1.00000000e-50,
           5.76271186e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   2.03389831e-02,
           1.00000000e-50,   1.00000000e-50],
        [  2.84280936e-01,   1.10367893e-01,   1.00000000e-50,
           1.00334448e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           2.20735786e-01,   1.00000000e-50,   1.00000000e-50,
           9.69899666e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00334448e-01,   1.77257525e-01,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   4.87804878e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           7.92682927e-01,   1.00000000e-50,   7.31707317e-02,
           8.53658537e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           5.26315789e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.57894737e-01,
           1.00000000e-50,   7.89473684e-01],
        [  3.28467153e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.02189781e-01,   1.82481752e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           2.48175182e-01,   1.00000000e-50,   1.38686131e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   3.12500000e-01,   1.00000000e-50,
           1.00000000e-50,   6.87500000e-01],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   9.00000000e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50]],

       [[  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   3.33333333e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           6.66666667e-01,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           2.30769231e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           6.15384615e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.53846154e-01],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   2.64900662e-02,   1.00000000e-50,
           1.00000000e-50,   9.73509934e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   8.31946755e-03,   3.76039933e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   6.15640599e-01],
        [  1.00000000e+00,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   8.34914611e-03,   1.00000000e-50,
           1.00000000e-50,   2.84629981e-03,   1.00000000e-50,
           6.54648956e-02,   9.14421252e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   5.69259962e-03,
           1.00000000e-50,   2.46679317e-03,   7.59013283e-04,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50]],

       [[  1.00000000e-50,   1.00000000e-50,   1.55820348e-02,
           1.00000000e-50,   6.59945005e-02,   3.84967919e-02,
           2.12648946e-01,   7.42438130e-02,   1.00000000e-50,
           5.31622365e-02,   1.00000000e-50,   1.00000000e-50,
           3.91384051e-01,   3.84967919e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   5.49954170e-03,
           1.04491292e-01,   1.00000000e-50],
        [  2.38379023e-03,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           8.70083433e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.27532777e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  5.83333333e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   3.33333333e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   8.33333333e-02,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e+00,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e+00,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.33333333e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   8.66666667e-01],
        [  1.00000000e-50,   6.15384615e-02,   1.00000000e-50,
           1.07692308e-01,   1.00000000e-50,   4.61538462e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           3.69230769e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50]],

       [[  1.00000000e-50,   6.65399240e-02,   2.07224335e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           8.17490494e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00760456e-01,   1.52091255e-02,
           4.94296578e-02,   4.79087452e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   7.36196319e-02,   1.00000000e-50,
           3.31288344e-01,   3.68098160e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.10429448e-01,   1.00000000e-50,   6.13496933e-03,
           1.00000000e-50,   1.00000000e-50,   8.58895706e-02,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           5.52147239e-02,   3.00613497e-01],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           4.34782609e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.73913043e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           3.91304348e-01,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   6.82046138e-02,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   2.60782347e-02,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           6.21865597e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   2.83851555e-01],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   2.82490975e-01,   1.85920578e-01,
           4.51263538e-03,   3.51985560e-02,   1.00000000e-50,
           6.31768953e-02,   1.00000000e-50,   1.00000000e-50,
           5.05415162e-02,   1.26353791e-02,   1.00000000e-50,
           3.42960289e-02,   2.40974729e-01,   1.00000000e-50,
           1.00000000e-50,   1.17328520e-02,   6.76895307e-02,
           1.08303249e-02,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e+00,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50]],

       [[  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.22166247e-01,   1.00000000e-50,
           3.12342569e-01,   4.84886650e-01,   1.00000000e-50,
           3.77833753e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           4.28211587e-02,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           5.90445518e-03,   1.00000000e-50,   2.68384326e-03,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           3.93630345e-03,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           9.87475398e-01,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   8.88888889e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.11111111e-01,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e+00],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           6.66666667e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   3.33333333e-01,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e+00,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.71052632e-01,   1.00000000e-50,   5.26315789e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           3.02631579e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50]],

       [[  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.76939529e-02,   7.77756173e-04,
           9.66945363e-01,   3.88878087e-03,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           4.66653704e-03,   1.00000000e-50,   1.00000000e-50,
           1.94439043e-04,   1.00000000e-50,   1.00000000e-50,
           5.83317130e-03,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           2.46406571e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   6.16016427e-03,   5.95482546e-02,
           9.09650924e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e+00,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   5.00000000e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   5.00000000e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e+00,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   8.75000000e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.25000000e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.38888889e-02,
           1.00000000e-50,   1.00000000e-50,   5.00000000e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   3.19444444e-01,   1.00000000e-50,
           1.00000000e-50,   1.66666667e-01]],

       [[  1.00000000e-50,   1.00000000e-50,   2.19512195e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           4.87804878e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.46341463e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.70731707e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   4.14634146e-01],
        [  1.00000000e-50,   2.95454545e-01,   1.00000000e-50,
           2.72727273e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   3.40909091e-01,   1.00000000e-50,
           9.09090909e-02,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e+00,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   2.53342716e-03,   1.68895144e-03,
           7.41731175e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.40745954e-04,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.09641098e-01,
           6.89655172e-03,   4.20830401e-02,   9.52850106e-02,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   4.58333333e-01,
           1.00000000e-50,   1.00000000e-50,   5.41666667e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   6.25000000e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   9.37500000e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50]],

       [[  1.00000000e-50,   2.38095238e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   3.01587302e-01,   1.00000000e-50,
           1.00000000e-50,   4.60317460e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   2.42236025e-01,   1.00000000e-50,
           4.34782609e-02,   1.00000000e-50,   6.21118012e-02,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           6.39751553e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.24223602e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.74146015e-02,   2.47823175e-02,   1.00000000e-50,
           1.00000000e-50,   2.88010717e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   8.70730074e-03,
           1.80843938e-02,   8.37240455e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           6.49698593e-02,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   2.27272727e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   3.63636364e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   4.09090909e-01],
        [  1.00000000e+00,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   8.03278689e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.96721311e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   5.46875000e-02,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           2.89062500e-01,   6.56250000e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50]],

       [[  5.18518519e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           3.88888889e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   9.25925926e-02],
        [  1.80327869e-01,   1.00000000e-50,   4.37158470e-02,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           6.63934426e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   3.55191257e-02,   6.28415301e-02,
           1.00000000e-50,   1.36612022e-02],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  5.00000000e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   5.00000000e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           5.88235294e-02,   1.00000000e-50,   1.17647059e-01,
           3.92156863e-02,   1.00000000e-50,   4.90196078e-01,
           1.00000000e-50,   2.94117647e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   4.40000000e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           2.80000000e-01,   1.00000000e-50,   2.80000000e-01,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   4.53514739e-04,   4.98866213e-03,
           6.34920635e-02,   1.00000000e-50,   2.66213152e-01,
           1.00000000e-50,   4.53514739e-04,   9.07029478e-04,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   4.98866213e-03,   1.21541950e-01,
           3.23356009e-01,   6.03174603e-02,   1.46031746e-01,
           1.00000000e-50,   7.25623583e-03]],

       [[  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           8.00000000e-01,   2.00000000e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   5.40540541e-02,   1.00000000e-50,
           1.00000000e-50,   4.05405405e-02,   1.00000000e-50,
           6.75675676e-02,   1.00000000e-50,   1.00000000e-50,
           5.67567568e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           2.70270270e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e+00,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   6.66666667e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   3.33333333e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  7.69230769e-02,   8.84615385e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   3.84615385e-02,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.30293160e-03,   1.00000000e-50,   1.00000000e-50,
           6.74267101e-02,   1.00000000e-50,   2.18241042e-02,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   9.09446254e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50]],

       [[  3.92857143e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.96428571e-01,
           4.10714286e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.20240481e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   8.45691383e-01,   1.02204409e-01,
           4.00801603e-02,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   5.72246066e-03,
           1.00000000e-50,   3.00429185e-02,   1.00000000e-50,
           1.00000000e-50,   7.01001431e-01,   2.01716738e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           6.15164521e-02,   1.00000000e-50],
        [  1.00000000e-50,   5.00000000e-01,   5.00000000e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   4.56431535e-02,   8.68257261e-01,
           1.00000000e-50,   2.38589212e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   2.90456432e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   3.31950207e-02,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e+00,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.56250000e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.25000000e-02,   1.00000000e-50,
           3.12500000e-02,   1.00000000e-50,   1.00000000e-50,
           5.50000000e-01,   1.00000000e-50,   1.81250000e-01,
           6.87500000e-02,   1.00000000e-50]],

       [[  1.73913043e-01,   1.00000000e-50,   2.17391304e-02,
           1.00000000e-50,   5.78804348e-01,   1.26358696e-01,
           1.00000000e-50,   1.08695652e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   5.43478261e-03,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   8.28804348e-02,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.88284519e-02,
           1.88284519e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.38075314e-01,   1.00000000e-50,   1.00000000e-50,
           7.92887029e-01,   1.00000000e-50,   1.00000000e-50,
           6.27615063e-03,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   2.51046025e-02],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   7.14285714e-02,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   6.42857143e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           2.85714286e-01,   1.00000000e-50],
        [  1.00000000e-50,   5.00000000e-01,   3.33333333e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.66666667e-01,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   5.00000000e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   5.00000000e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.71428571e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   8.57142857e-02,   7.14285714e-01,
           2.85714286e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   4.50000000e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.75000000e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   3.75000000e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50]],

       [[  1.00000000e-50,   1.00000000e+00,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e+00,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   8.13559322e-01,
           1.00000000e-50,   1.00000000e-50,   3.38983051e-02,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.52542373e-01,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   3.37268128e-04,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   9.92580101e-01,
           1.00000000e-50,   6.91399663e-03,   1.68634064e-04,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   6.86032138e-01,   1.00000000e-50,
           2.74412855e-01,   2.22496910e-02,   1.00000000e-50,
           1.00000000e-50,   1.73053152e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   6.50793651e-01,
           1.00000000e-50,   3.49206349e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50]],

       [[  1.00000000e-50,   1.00000000e-50,   3.33333333e-01,
           1.00000000e-50,   1.00000000e-50,   1.66666667e-01,
           5.00000000e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.06382979e-01,
           1.91489362e-01,   3.19148936e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.91489362e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           4.78723404e-01,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   4.78468900e-02,   1.00000000e-50,
           1.00000000e-50,   3.50877193e-02,   9.01116427e-01,
           1.59489633e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   2.67379679e-03,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           2.00534759e-03,   1.00000000e-50,   5.96256684e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   3.99064171e-01],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   3.17073171e-01,
           1.00000000e-50,   6.82926829e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           9.52380952e-03,   1.00000000e-50,   1.00000000e-50,
           6.19047619e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           9.09523810e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.90476190e-02]],

       [[  1.00000000e-50,   1.92307692e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           3.84615385e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   7.69230769e-02,   1.00000000e-50,
           1.00000000e-50,   1.92307692e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.53846154e-01,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.81818182e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           4.54545455e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           3.63636364e-01,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.03270224e-02,   2.75387263e-02,   1.00000000e-50,
           1.00000000e-50,   5.33562823e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   3.09810671e-02,
           1.91049914e-01,   1.00000000e-50,   1.00000000e-50,
           6.86746988e-01,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.18279570e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.89964158e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   6.91756272e-01],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  2.45775730e-02,   1.00000000e-50,   3.53302611e-02,
           1.00000000e-50,   3.99385561e-01,   6.45161290e-02,
           9.83102919e-02,   1.00000000e-50,   2.30414747e-02,
           6.14439324e-02,   1.00000000e-50,   4.45468510e-02,
           1.38248848e-02,   2.18125960e-01,   1.00000000e-50,
           4.60829493e-03,   1.22887865e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50]],

       [[  1.00000000e-50,   7.85714286e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.28571429e-01,
           1.00000000e-50,   6.85714286e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   7.85714286e-02,   1.00000000e-50,
           1.00000000e-50,   2.85714286e-02],
        [  1.00000000e-50,   2.76315789e-01,   5.78947368e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.44736842e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   3.86138614e-01,   1.00000000e-50,
           1.00000000e-50,   7.92079208e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           3.51485149e-01,   1.00000000e-50,   1.83168317e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.42857143e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           6.92307692e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.64835165e-01],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e+00,
           1.00000000e-50,   1.00000000e-50],
        [  1.37477585e-02,   1.55409444e-02,   1.00000000e-50,
           1.00000000e-50,   1.08188882e-01,   2.95277944e-01,
           5.61864913e-02,   2.62402869e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.88882247e-01,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   3.34728033e-02,
           1.00000000e-50,   2.63000598e-02],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e+00,   1.00000000e-50]],

       [[  1.00000000e+00,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           5.52486188e-02,   1.00000000e-50,   1.00000000e-50,
           3.31491713e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   2.65193370e-01,   2.15469613e-01,
           1.32596685e-01,   1.00000000e-50],
        [  1.00000000e-50,   8.89521437e-04,   1.42323430e-03,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   3.46913361e-02,   9.61038961e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.95694716e-03,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.60714286e-01,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   8.39285714e-01],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   8.33333333e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   5.83333333e-01,   1.00000000e-50,
           1.00000000e-50,   3.33333333e-01],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e+00,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   5.51601423e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   2.02846975e-01,
           1.00000000e-50,   5.33807829e-03,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           7.36654804e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50]],

       [[  2.35849057e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           3.77358491e-02,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   6.60377358e-01,   6.60377358e-02,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.81818182e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           4.54545455e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           3.63636364e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   5.57142857e-01,   1.42857143e-02,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           4.28571429e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           7.41935484e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   2.58064516e-01],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.38989170e-01,
           1.00000000e-50,   1.80505415e-03,   1.38387485e-02,
           1.20336943e-02,   1.50421179e-02,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           2.10589651e-02,   1.81107100e-01,   5.94464501e-01,
           1.14320096e-02,   1.02286402e-02],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           3.80952381e-01,   1.00000000e-50,   1.00000000e-50,
           6.19047619e-01,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50],
        [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
           1.00000000e-50,   1.00000000e-50]]])

#######################################
# Transition probabilities angle node #
#######################################

# Distribution for angle node in the first slice

angleId0_cpd = \
array([  1.00000000e+00,   1.00000000e-50,   1.00000000e-50,
         1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
         1.00000000e-50])

# The transition matrix for the angle node in the following slices

angleId1_cpd = \
array([[  1.00000000e-50,   1.00000000e+00,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50],
       [  1.00000000e-50,   1.00000000e-50,   1.00000000e+00,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50],
       [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e+00],
       [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e+00,   1.00000000e-50,
          1.00000000e-50],
       [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e+00,
          1.00000000e-50],
       [  1.00000000e+00,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50],
       [  1.00000000e-50,   1.00000000e-50,   1.00000000e-50,
          1.00000000e+00,   1.00000000e-50,   1.00000000e-50,
          1.00000000e-50]])


In [7]:
########## FUNCTION FOR MAKING DATA AND MISMASKS ##########
angle_ids = [0.0, 1.0, 2.0, 6.0, 3.0, 4.0, 5.0]

def make_data_and_mism(len_nucs):
    """
    Make a sequence and a mismask for model of length 'len_nucs'
    """
    len_angles = len_nucs*num_angles

    data = zeros((len_angles, num_nodes))
    mism = ones((len_angles, num_nodes))

    # Set the angle IDs
    data[:,angle_id_pos] = array(angle_ids*len_nucs)
    mism[:,angle_id_pos] = 0

    return data, mism

########## FUNCTION FOR CONVERTING DATA INTO AN ANGLELIST ##########

#def data_to_anglelist(data):
#    length = data.shape[0]
#    angle_list = [tuple(map(float, list(l))) for l in data[:,angle_pos].reshape((length//7,7))[:,(0,1,2,4,5,6,3)]]
#    return print(angle_list)
def data_to_anglelist(data):
    length = data.shape[0]
    angle_list = [tuple(map(float, list(l))) for l in data[:,angle_pos].reshape((length//7,7))[:,(0,1,2,4,5,6,3)]]
    return angle_list



In [8]:
def full_atom_name(name):
    if len(name) <= 3:
        return " %-3s" % name
    elif len(name) == 4:
        return name
    else:
        raise Exception("Atom names can be no longer than 4 characters!")


def sAtom_pre_1_53(name, vector):
    """
    A wrapper function for creating a simple atom for Bio.PDB pre version 1.53.
    """
    return Atom.Atom(name, vector.get_array(), 0, 1, " ", full_atom_name(name), None)


def sAtom(name, vector):
    """
    A wrapper function for creating a simple atom for Bio.PDB post version 1.53.
    """
    full_name = full_atom_name(name)
    atom = Atom.Atom(name, vector.get_array(), 0, 1, " ", full_name, None, name[0])
    return atom

In [9]:
class SamplerFwBt:
    """
    Sampler for a multiple-input/multiple-output HMM with only one
    hidden DISCRETE node using forward backtrack.
    """
    def __init__(self, dbn, hidden_node):
        # Some copies for persistence.
        self.dbn=dbn

        # List of node objects
        nodes_0=self.nodes_0=dbn.nodes_0
        nodes_1=self.nodes_1=dbn.nodes_1
        self.nr_nodes=dbn.nr_nodes
        self.total_output_size=len(self.nodes_0)

        self.hidden_node = hidden_node

        # The position index in the cpd of the hidden node in slice 1
        # for the hidden node in the previous slice
        self.hidden_node_index_cpd = self.nodes_1[hidden_node].parents_0.index(hidden_node)
        self.hidden_node_size = self.nodes_1[hidden_node].node_size
    
    def sweep(self, mismask, start, end):
        # Make some local copies for speed.
        nodes_0 = self.nodes_0
        nodes_1 = self.nodes_1
        nr_nodes = self.nr_nodes
        hidden_node = self.hidden_node
        hidden_node_size = self.hidden_node_size
        seq_len = len(mismask)
        hnic = self.hidden_node_index_cpd

        assert(nodes_0[hidden_node].node_size==nodes_1[hidden_node].node_size)
        node_size = int(nodes_0[hidden_node].node_size)

        slice_count=(end-start)
        trans_probs = []

        # If the whole sequence has to be sampled, it is sufficient to
        # first sample the hidden nodes in order and then the output
        # nodes (since there are no observed output nodes for
        # BARNACLE)
        if start==0 and end==seq_len:
            for l in range(0, seq_len):
                if l==0:
                    node=nodes_0[hidden_node]
                else:
                    node=nodes_1[hidden_node]

                # Sample the hidden node
                node.sample(l)

                # Sample the children of the hidden node
                for child in node.children_1:
                    child.sample(l)

            return slice_count

        # Calculate the forward array
        forward = zeros((slice_count, node_size))
        for i,l in enumerate(range(start, end)):
            # For the first slice
            if l==0 or i==0:
                if l==0:
                    node=nodes_0[hidden_node]
                else:
                    node=nodes_1[hidden_node]

                # Get the values of the parrents
                pv=tuple(node.parentmap[l][:-1])

                # Get the transistion probability (and cache it for backtrack)
                trans_prob = node.cpd[pv]
                trans_probs.append(trans_prob)

                forward[i] = node.cpd[pv]

            # For all following slices
            else:
                node=nodes_1[hidden_node]

                # Get the values of the parents except the value of the hidden node in previous slice
                pv = list(node.parentmap[l][:-1])
                pv[hnic] = slice(None)
                pv = tuple(pv)

                # Get the transistion probability (and cache it for backtrack)
                trans_prob = node.cpd[pv]
                trans_probs.append(trans_prob)

                # Multiply transistionsvalues g->h by the
                # forwardvalues [i-1, g] and sum over nodevalues of g
                forward[i] = sum(transpose(trans_prob)*forward[i-1], axis=1)

            # Note, we do not multiply the forward values by the
            # probability of the children, since there are no observed
            # children in BARNACLE.

            # If the is the end slice but not the end of the DBN, take the next hidden value into account
            if l==end-1 and l!=seq_len-1:
                node_next = nodes_1[hidden_node]
                pv_next = list(node_next.parentmap[l+1])
                pv_next[hnic] = slice(None)
                forward[i] *= node_next.cpd[tuple(pv_next)]
                                    
            # Scale forward to be a probability distribution (as
            # described in 'Biological Sequence Analysis' by R Durbin
            # et al., p. 78, Cambridge University Press, 1998.
            s=sum(forward[i])
            forward[i]/=s
                    
        # Do the backtrack
        for l in range(end-1, start-1,-1):
            i = l-start

            if l==0:
                node=nodes_0[hidden_node]
            else:
                node=nodes_1[hidden_node]

            # Choose whether to sample from the forward probability
            # distribution or take the previous choice into account
            if l==end-1:
                distribution = forward[i]
            else:
                proportional = forward[i] * trans_probs[i+1][:,choice]
                distribution = proportional / sum(proportional)

            # Sample the hidden node from the forward probability distribution
            cum_forward = cumsum(distribution)
            r=random()
            choice=bisect(cum_forward, r)
            node.parentmap[l]=choice

            # Sample the children
            for child in node.children_1:
                child.sample(l)

        return slice_count

In [10]:
class Node:
    """
    Abstract base class for all L{Node} objects.
    """
    def __init__(self, output_size, node_type):
        """
        @param output_size: dimension of output vector
        @type output_size: int

        @param node_type: type of node (ie. DISCRETE)
        @type node_type: string
        """
        self.output_size=output_size
        # Number of values the node can adopt
        # Children in same slice
        self.children_1=[]
        # Children in next slice
        self.children_2=[]
        # Parent indices in previous slice
        self.parents_0=[]
        # Sizes of parents in previous slice
        self.parents_0_sizes=[]
        # Parents indices in same slice
        self.parents_1=[]
        # Sizes of parents in same slice
        self.parents_1_sizes=[]
        self.fixed=0
        self.is_constructed=0
        self.node_type=node_type

    # Public

    def set_node_index(self, ni):
        """
        Set the index of the node in the node list.

        @param ni: index of node (ie. 0=first node in slice)
        @type ni: int
        """
        self.node_index=ni

    def set_data_index(self, di):
        """
        Set the index of where to find the node data in the 
        sequence data.

        @param di: index of node data in a slice
        @type di: int
        """
        self.data_index=di

    def add_intra_child(self, node):
        """
        Add an intra-slice child.

        @param node: child node (same slice) of current node 
        @type node: L{Node}
        """
        self.children_1.append(node)

    def add_inter_child(self, node):
        """
        Add a child in the previous slice.

        @param node: child node (previous slice) of current node
        @type node: L{Node}
        """
        self.children_2.append(node)

    def add_inter_parent(self, data_index, node_size):
        """
        Add a parent in the previous slice.
        
        @param data_index: index of parent value in previous slice
        @type data_index: int

        @param node_size: output size of parent
        @type node_size: int
        """
        self.parents_0.append(data_index)
        self.parents_0_sizes.append(node_size)

    def add_intra_parent(self, data_index, node_size):
        """
        Add an intra-slice parent.

        @param data_index: index of parent value in same slice
        @type data_index: int

        @param node_size: output size of parent
        @type node_size: int
        """
        self.parents_1.append(data_index)
        self.parents_1_sizes.append(node_size)

    def get_parentmap(self, seq, weight):
        """
        Translate a sequence into a FamilyMap and return 
        the FamilyMap object. The FamilyMap object is a
        cache for fast parent and child lookup.

        @param seq: data sequence
        @type seq: numpy array

        @return: family map data structure which implements fast child
            and parent value lookup
        @rtype: FamilyMap object
        """
        raise NotImplementedError

    def set_parentmap(self, parentmap):
        """
        Load a L{ParentMap} object into the  L{Node} object. The L{ParentMap} object 
        is a cache for fast parent and child value lookup. Loading a 
        L{ParentMap} object thus means loading new sequence data.

        @param parentmap: load a L{ParentMap} object into the L{Node} object
        @type parentmap: L{ParentMap}
        """

        self.parentmap=parentmap
        self.weight=parentmap.get_weight()
        self.seq_len=len(parentmap)

    def construct(self, slice):
        """
        Initialize the data structures of the node.
        'slice' indicates to which slice the node belongs
        (ie. slice l=0 or slice l>0).

        @param slice: slice (0=slice at l=0, 1=slice at l>0)
        @type slice: int
        """
        if self.is_constructed==1:
            raise MocapyException("a Node should only be constructed once.")
        self.slice=slice
        self.is_constructed=1

    def sample(self, l):
        """
        Sample the node at slice l. 
        This is done by using P(node|parents), ie. not 
        taking the children into account.
        
        @param l: sequence position
        @type l: int
        """
        raise NotImplementedError

    def update_ess(self, family_values):
        """
        After sampling a hidden node, the ESS are updated using
        this function. This is done using the values of the node itself and its
        parents (ie. the family values).
        
        @param family_values: values of parents and node itself
        @type family_values: numpy array
        """
        raise NotImplementedError

    def save_ess(self):
        """
        Calculate the ESS for a whole sequence, save it, and re-initialize
        the ESS data structure. This function is called after a whole sequence
        has been sampled.
        """ 
        raise NotImplementedError

    def blanket_sample(self, l):
        """
        Sample the node value at slice l based on its
        Markov blanket. The Markov blanket consists of
        the parent and child nodes of the node.

        @param l: sequence position
        @type l: int
        """
        raise NotImplementedError

    def get_slice_log_likelihood(self, l):
        """
        Return the log likelihood of the node in slice l.
        @param l: sequence position
        @type l: int

        @return: log likelihood of nodes in slice l
        @rtype: float
        """
        raise NotImplementedError

    def get_ess_list(self):
        """
        Return the list of ESS values.
        This method is used to gather all ESS calculated
        on different nodes when running in parallel. The
        ESS are put in one list, and passed on to the 
        'do_M_step' method.

        @return: list of ESS
        @rtype: list
        """
        raise NotImplementedError

    def do_M_step(self, ess_list):
        """
        Update the parameters of the node using the ESS values.

        @param ess_list: list of ESS
        @type ess_list: list
        """
        raise NotImplementedError

    def get_parameters(self):
        """
        Return the relevant parameters of the node.
        """
        raise NotImplementedError

    def fix(self, flag):
        """
        Fix the node (flag=1), or unfix the node (flag=0).

        @param flag: fix flag
        @type flag: int (0 or 1)
        """
        self.fixed=flag




In [11]:
StringType = type(str)

to_one_letter_code = {'  A':'A', '  C':'C', '  G':'G', '  T':'T','  U':'U'}
to_three_letter_code = {'A':'  A', 'C':'  C', 'G':'  G', 'T':'  T','U':'  U'}
purine_one_letter_codes = ['A','G']
pyrimidine_one_letter_codes = ['C','T','U']
rna_one_letter_codes = ['A','C','G','U']
rna_three_letter_codes = ['  A','  C','  G','  U']

def get_one_letter_code(residue):
    """
    Returns one letter code for a residue or converts a three letter
    code to a one letter code.
    
    @param residue: a L{Residue} object OR the three letter nucleic acid code
    @type residue: L{Residue} or string
    @return: one letter code (in UPPER)
    @rtype: L{string}

    """
    if not type(residue)==StringType:
        residue=residue.get_resname()
    residue=residue.upper()
    if residue in to_one_letter_code:
        return to_one_letter_code[residue]
    else:
        return None

def get_three_letter_code(residue):
    """
    Returns three letter code for a residue or converts a one letter
    code to a three letter code.
    
    @param residue: a L{Residue} object OR the three letter nucleic acid code
    @type residue: L{Residue} or string
    @return: three letter code (in UPPER)
    @rtype: L{string}

    """
    if not type(residue)== str:
        return residue.get_resname().upper()
    else:
        residue.upper()
        if residue in to_three_letter_code:
            return to_three_letter_code[residue]
        else:
            return "  X"



def is_rna_nuc(residue):
    """
    Return 1 if residue object/string is an RNA nucleotide.

    @param residue: a L{Residue} object OR the one letter nucleic acid code
    @type residue: L{Residue} or string
    @return: True/False
    @rtype: L{bool}
    """
    return (get_one_letter_code(residue) in rna_one_letter_codes)  


def is_purine(residue):
    """
    Return 1 if residue object/string is a purine

    @param residue: a L{Residue} object OR the one letter nucleic acid code
    @type residue: L{Residue} or string
    @return: True/False
    @rtype: L{bool}
    """
    return (get_one_letter_code(residue) in purine_one_letter_codes)


def is_pyrimidine(residue):
    """
    Return 1 if residue object/string is a pyrimidine

    @param residue: a L{Residue} object OR the one letter nucleic acid code
    @type residue: L{Residue} or string
    @return: True/False
    @rtype: L{bool}
    """
    return (get_one_letter_code(residue) in pyrimidine_one_letter_codes)


def has_base_type(type,residue):
    """
    Return 1 if residue object has base type 'type' 

    @param type: the one letter nucleic acid code (in UPPER)
    @type type: string
    @param residue: a L{Residue} object
    @type residue: L{Residue}
    @return: True/False
    @rtype: L{bool}
    """
    return (get_one_letter_code(residue) == type)

In [12]:
def atoms_to_residues(atoms, atomsPerRes, seqid, resnames=None):
    """
    Builds a list of residues from a list of atoms, where each residue
    i the list has 'atomsPerRes' atoms per residue. All residues are
    set to 'A', if no residue names are given.
    """

    # Check the arguments
    if (len(atoms) % atomsPerRes) != 0:
        raise ValueError("Mismatch between list length and atoms per residues.")

    if resnames==None:
        resnames = ["  A" for _ in range(len(atoms)/atomsPerRes)]


    residues = []
    res = None
    
    for (i,atom) in enumerate(atoms):
        # Residue number
        resnumber = i // atomsPerRes

        # Create a new residue
        if (i % atomsPerRes) == 0:
            if res!=None:
                residues.append(res)
            res = Residue((" ", resnumber, " "), resnames[resnumber], seqid)
            res.add(atom)
        # Add an atom
        else:
            res.add(atom)

    if res!=None:
        residues.append(res)

    return residues


def residues_to_struct(name_residues_list, structID):
    """
    Build a structure from a list of (chain name, residues) - each as
    a chain.
    """
    
    builder = StructureBuilder()
    builder.init_structure(structID)
    builder.init_model(0)

    for (name, residues) in name_residues_list:
        builder.init_chain(name)
        for res in residues:
            builder.chain.add(res)

    return builder.get_structure()


def convert_from_bb_dihedrals(dihedrals_list, check_valid_dihedrals=True):
    """
    Construct the RNA backbone from a list of tuples (alpha, beta,
    gamma, delta, epsilon, zeta) of backbone dihedrals
    """ 
    if check_valid_dihedrals:
        # Check that the list is correct (only the first tuple may start with none and the last may end with two nones)
        if None in dihedrals_list[0][1:] or None in dihedrals_list[-1][:-2] or \
               (len(dihedrals_list)>2 and [tup for tup in dihedrals_list[1:-1] if None in tup] != []):
            raise ValueError("The list of dihedrals is not valid")

    # List of Residues that is the result
    atoms = []

    # Setup the angles and distances to use
    atomnames = bb_atomnames

    # Use different angles and distances based on delta [3]
    if dihedrals_list[0][3] < 1.8:
        distances = bb_distances_3e
        angles = bb_angles_3e
    else:
        distances = bb_distances_2e
        angles = bb_angles_2e

    # Start in the origin and place the two next atoms in the x-y-plane
    P  = sAtom(atomnames[0], Vector(0,0,0))
    O5 = sAtom(atomnames[1], Vector(distances[0], 0, 0))
    C5 = sAtom(atomnames[2], Vector(distances[0], 0, 0) + Vector(cos(pi-angles[1]), sin(pi-angles[1]), 0)**distances[1])

    atoms = [P, O5, C5]

    n = len(atomnames)

    # Loop invariant:
    #  - Start: the list consists of whole sets of P-O5-C
    #  - During: last pair is CA-C-N
    #  - End: the list consists of whole sets of N-CA-C
    for (i, dihedrals) in enumerate(dihedrals_list):
        
        # Make sure that dihedrals[0][0], dihedrals[-1][-2] and dihedrals[-1][-1] are None
        if i==0 and dihedrals[0]!=None:
            dihedrals = list(dihedrals)
            dihedrals[0] = None
        if i==len(dihedrals_list)-1 and (dihedrals[-2]!=None or dihedrals[-1]!=None):
            dihedrals = list(dihedrals)
            dihedrals[-2] = dihedrals[-1] = None

        # Use different angles and distances based on delta [3]
        if dihedrals[3] < 1.8:
            distances = bb_distances_3e
            angles = bb_angles_3e
        else:
            distances = bb_distances_2e
            angles = bb_angles_2e

        # Place the atoms
        for (j, dihedral) in enumerate(dihedrals):
            if dihedral!=None:
                A, B, C = atoms[-3:]
                D = fromDihedral(A.get_vector(), B.get_vector(), C.get_vector(), \
                                 distances[j%n], distances[(j+1)%n], angles[(j+1)%n], dihedral)
                atoms.append(sAtom(atomnames[(j+2)%n], D))

    return atoms


def convert_from_dihedrals(dihedrals_list, sequence=None, bb_atoms=None, accept_no_solution=True,
                           add_sugar=True, add_base=True):
    """
    Construct a list of residues from a list of tuples of dihedral angles in the order:
    [(alpha, beta, gamma, delta, epsilon, zeta, chi), (alpha, beta ..., zeta, chi), ...]

    The angles must be in radians.

    @param dihedral_list: the list of tuples of dihedral angles.
    @type dihedral_list: L{(float, float, float, float, float, float, float)}

    @param sequence: optional string of nucleic acids. If set to None
                     all residues will be of type A
    @type sequence: string or None

    @param bb_atoms: optional list of back bone atoms. If this list is
                     given, the back bone will not be constructed from
                     the dihedral angles given.
    @type bb_atoms: L{Atom} or None
    
    @param accept_no_solution: If this option is set to false, the
                               function will raise an exception if the
                               method fails to find a solution for the
                               coordinates of C1'. This could indicate
                               that the dihedral angles given as
                               argument are 'bad'. If the options is
                               set to true, no exception will be
                               raised.
    @type accept_no_solution: bool

    @param add_sugar: indicated if the atoms in the sugar ring should be placed.
    @type add_sugar: bool

    @param add_base: indicated if the atoms in the base should be
                     placed. Ineffective if add_sugar is false
    @type add_base: bool

    @return: A list of residues
    @rtype: L{Resudue}
    
    """ 
    
    # Construct the backbone
    #print(bb_atoms)
    #print(type(bb_atoms))
    if bb_atoms == None:
        bb_dihedrals = [tup[:6] for tup in dihedrals_list]
        bb_atoms = convert_from_bb_dihedrals(bb_dihedrals)

    if sequence!=None:
        sequence = list(map(get_three_letter_code, list(sequence)))

    residues = atoms_to_residues(bb_atoms, 6, "0", resnames=sequence)
   
    # Construct each sugar and base
    for (resnumber, res) in enumerate(residues):

        # Attached oxygens OP1 and OP2 to the backbone P
        if resnumber!=0:
            O3p_prev = residues[resnumber-1]["O3'"].get_vector()
            P   = res["P"].get_vector()
            O5p = res["O5'"].get_vector()

            O1P = from3atoms(P, O3p_prev, O5p, a_O3pPO1P, a_O5pPO1P, d_PO1P, 0)
            O2P = from3atoms(P, O3p_prev, O5p, a_O3pPO2P, a_O5pPO2P, d_PO2P, 1)

            res.add(sAtom("OP1", O1P))
            res.add(sAtom("OP2", O2P))

        # Place the atoms in the sugar ring
        if add_sugar:
            
            # Make reference to the vectors of atom that will be used
            C5p = res["C5'"].get_vector()
            C4p = res["C4'"].get_vector()
            C3p = res["C3'"].get_vector()
            O3p = res["O3'"].get_vector()

            # Use different angles and distances based on delta [3]
            if dihedrals_list[resnumber][3] < 1.8:
                a_C5pC4pO4p = a_3e_C5pC4pO4p
                a_C3pC4pO4p = a_3e_C3pC4pO4p
                a_C2pC3pC4p = a_3e_C2pC3pC4p
                a_C2pC3pO3p = a_3e_C2pC3pO3p
                a_C4pO4pC1p = a_3e_C4pO4pC1p
                a_C1pC2pC3p = a_3e_C1pC2pC3p
                a_O4pC1pN   = a_3e_O4pC1pN
                a_C2pC1pN   = a_3e_C2pC1pN
                d_C4pO4p    = d_3e_C4pO4p
                d_C2pC3p    = d_3e_C2pC3p
                d_O4pC1p    = d_3e_O4pC1p
                d_C1pC2p    = d_3e_C1pC2p
                d_C1pN      = d_3e_C1pN

                a_C1pC2pO2p = a_3e_C1pC2pO2p
                a_C3pC2pO2p = a_3e_C3pC2pO2p
                d_C2pO2p    = d_3e_C2pO2p


            else:
                a_C5pC4pO4p = a_2e_C5pC4pO4p
                a_C3pC4pO4p = a_2e_C3pC4pO4p
                a_C2pC3pC4p = a_2e_C2pC3pC4p
                a_C2pC3pO3p = a_2e_C2pC3pO3p
                a_C4pO4pC1p = a_2e_C4pO4pC1p
                a_C1pC2pC3p = a_2e_C1pC2pC3p
                a_O4pC1pN   = a_2e_O4pC1pN
                a_C2pC1pN   = a_2e_C2pC1pN
                d_C4pO4p    = d_2e_C4pO4p
                d_C2pC3p    = d_2e_C2pC3p
                d_O4pC1p    = d_2e_O4pC1p
                d_C1pC2p    = d_2e_C1pC2p
                d_C1pN      = d_2e_C1pN

                a_C1pC2pO2p = a_2e_C1pC2pO2p
                a_C3pC2pO2p = a_2e_C3pC2pO2p
                d_C2pO2p    = d_2e_C2pO2p

            # From C5p,C4p and C3p place 04p
            O4p = from3atoms(C4p,C5p,C3p, a_C5pC4pO4p, a_C3pC4pO4p, d_C4pO4p,0) # Always solution 0
            res.add(sAtom("O4'", O4p))

            # From C4p,C3p and O3p place C2p
            C2p = from3atoms(C3p,C4p,O3p, a_C2pC3pC4p, a_C2pC3pO3p, d_C2pC3p,1) # Always solution 1
            res.add(sAtom("C2'", C2p))

            # From O4p and C2p place C1
            try:
                C1p = from4atoms(O4p,C2p,C4p,C3p,a_C4pO4pC1p,a_C1pC2pC3p,d_O4pC1p,d_C1pC2p, accept_no_solution=accept_no_solution)
            except ValueError as exception:
                raise ConversionException(0, resnumber)
            res.add(sAtom("C1'", C1p))

            # Attached oxygen O2' to C2' in the sugar ring
            O2p = from3atoms(C2p, C1p, C3p, a_C1pC2pO2p, a_C3pC2pO2p, d_C2pO2p, 1)
            res.add(sAtom("O2'", O2p))

            # Place the atoms in the base 
            if add_base:

                # From O4p,C1p and C2p place N
                N = from3atoms(C1p,O4p,C2p, a_O4pC1pN, a_C2pC1pN, d_C1pN,0)         # Always solution 0
                if is_purine(res):
                    res.add(sAtom("N9", N))
                else:
                    res.add(sAtom("N1", N))

                # From C2p,C1p,N and chi place C2/C4 and base
                if has_base_type("A", res):
                    d_NC = d_A_N9C4
                    a_CNC1p = a_A_C4N9C1p
                    base_dict, base_coords, base_names = A_dict, A_coords, A_names
                elif has_base_type("C", res):
                    d_NC = d_C_N1C2
                    a_CNC1p = a_C_C2N1C1p
                    base_dict = C_dict
                    base_dict, base_coords, base_names = C_dict, C_coords, C_names
                elif has_base_type("G",res):
                    d_NC = d_G_N9C4
                    a_CNC1p = a_G_C4N9C1p
                    base_dict, base_coords, base_names = G_dict, G_coords, G_names
                elif has_base_type("U",res):
                    d_NC = d_U_N1C2
                    a_CNC1p = a_U_C2N1C1p
                    base_dict, base_coords, base_names = U_dict, U_coords, U_names
                else:
                    print("Warning: Unsupported base, %s, met" % get_one_letter_code(res))

                chi = dihedrals_list[resnumber][6]
                if chi==None:
                    raise ValueError("The list of chis is not valid")

                C24 = fromDihedral(O4p, C1p, N, d_C1pN, d_NC, a_CNC1p, chi)

                # Do the actual base placing
                if is_purine(res):
                    base = place_base(N, C24, C1p, base_dict["N9"], base_dict["C4"], base_dict["C1'"], base_coords[2:])
                else:
                    base = place_base(N, C24, C1p, base_dict["N1"], base_dict["C2"], base_dict["C1'"], base_coords[2:])

                base_atom_names = base_names[2:]

                for (i,atom) in enumerate(base):
                    res.add(sAtom(base_atom_names[i], atom))
  
    return residues

In [13]:
# Set constant values
num_angles = 7
h_size = 20

angle_id_pos = 0
hd_pos = 1
angle_pos = 2
num_nodes = 3


def make_model():
    """
    Method for constructing the model
    """

    ## Specification of the nodes in the DBN

    # Angle identifier - Slice 0 and remaining slices
    angleId0 = DiscreteNode(node_size=num_angles, name='angleId0', user_cpd=angleId0_cpd)
    angleId1 = DiscreteNode(node_size=num_angles, name='angleId1', user_cpd=angleId1_cpd)

    # Hidden node - Slice 0 and remaining slices
    hd0 = DiscreteNode(node_size=h_size, name='hd0', user_cpd=hd0_cpd)
    hd1 = DiscreteNode(node_size=h_size, name='hd1', user_cpd=hd1_cpd)

    # Angle node
    angle = VMNode(node_size=h_size, name='angle', user_mus=angle_mus, user_kappas=angle_kappas)

    # Nodes in the first slice
    start_nodes = [angleId0, hd0, angle]

    # Nodes in the remaining slices
    end_nodes = [angleId1, hd1, angle]


    ## Construct the DBN
    dbn = DBN(start_nodes, end_nodes)

    # Connections internally in the slices
    dbn.add_intra('angleId0', 'hd0')
    dbn.add_intra('hd0','angle')

    # Connections between consecutive slices
    dbn.add_inter('hd0','hd1')
    dbn.add_inter('angleId0','angleId1')

    dbn.construct()

    return dbn

In [14]:
one = 0.99

# Likelohood calculator for IOHMM using the Forward algorithm
class LikelohoodIOHMM_Fw:
    """
    Likelohood calculator for IOHMM using the Forward algorithm
    """
    def __init__(self, dbn, hidden_node):
        self.dbn=dbn
        # List of node objects
        nodes_0=self.nodes_0=dbn.nodes_0
        nodes_1=self.nodes_1=dbn.nodes_1

        self.hidden_node = hidden_node
        self.hidden_node_index_cpd = 0
        self.hidden_node_size = nodes_1[hidden_node].node_size

        self.forward = None
        self.parentmap_list = None

        # Assert that the hidden node has same size in both slices
        assert(nodes_0[hidden_node].node_size==nodes_1[hidden_node].node_size)

    def set_data_in_dbn(self, seq):
        dbn = self.dbn
        parentmap_list = ([],[])

        # Put the sequence into the dbn
        for node in dbn.unique_nodes:
            parentmap=node.get_parentmap(seq, weight=1)
            node.set_parentmap(parentmap)
            parentmap_list[0].append(parentmap)

    def calc_ll(self, seq, mismask, start=None, end=None, ignore_child_mism=True, set_data_in_dbn=True):
        # Make some local copies for speed.
        dbn = self.dbn
        nodes_0 = self.nodes_0
        nodes_1 = self.nodes_1
        hidden_node = self.hidden_node
        hnic = self.hidden_node_index_cpd
        hidden_node_size = self.hidden_node_size

        # Assert that seq and mismask has same shape
        assert(seq.shape == mismask.shape)

        # Put the data into the dbn
        if set_data_in_dbn:
            self.set_data_in_dbn(seq)
        
        # Set the start- and end-variable correct
        if start == None:
            start = 0

        if end == None:
            end = seq.shape[0]

        # Set the rest of the variables
        slice_count = (end-start)
        scales = zeros(slice_count)

        # Calculate the forward array
        forward = zeros((slice_count, hidden_node_size))
        for i,l in enumerate(range(start, end)):
            # For the first slice
            if l==0 or i==0:
                if l==0:
                    node=nodes_0[hidden_node]
                    nodes = nodes_0
                else:
                    node=nodes_1[hidden_node]
                    nodes = nodes_1

                # Get the values the parents
                pv = tuple(node.parentmap[l][:-1])

                # Get the transistion probability
                trans_prob = node.cpd[pv]
                forward[i] = trans_prob
            
            # For all following slices
            else:
                node=nodes_1[hidden_node]
                nodes = nodes_1

                # Get the values the parents
                pv = list(node.parentmap[l][:-1])

                # Slice out the previous value of the hidden node
                pv[hnic] = slice(None)
                pv = tuple(pv)

                # Get the transistion probability
                trans_prob = node.cpd[pv]

                # Multiply transistionsvalues g->h by the
                # forwardvalues [i-1, g] and sum over nodevalues of g
                forward[i] = sum(transpose(trans_prob)*forward[i-1], axis=1)

            # Multiply the forward values by the probability of the observed children
            # Note that the value of the hidden node is preserved in prev_node_value
            prev_node_value = node.parentmap[l][-1]
            for child in node.children_1:
                if ignore_child_mism or mismask[l,child.node_index] < one:
                    for j in range(hidden_node_size):
                        node.parentmap[l] = j
                        forward[i,j] *= exp( child.get_slice_log_likelihood(l)[0] )
            node.parentmap[l] = prev_node_value

            # Scale forward to be a probability distribution (this is
            # allowed according to BSA sec. 3.6 (p. 78)
            s=sum(forward[i])
            forward[i]/=s
            scales[i] = s


        self.forward = forward

        return sum(log(scales)), slice_count




In [15]:
class ConversionException(Exception):
    def __init__(self, value, position=None):
        self.value = value
        self.position = position
        
    def __str__(self):
        return ("ReconstructException:" + repr(self.value))

    def get_position(self):
        return self.position


def fromDihedral(A, B, C, bond_BC, bond_CD, angle_BCD, torsion_BC):
    """
    Calculate a new point from three points A,B,C, an dihedral angle
    (torsion_BC) and bond angle (angle_BCD). Done by putting a
    coordinate system in C and then going from spherical coordinates
    and then translating the coordinate system to C.

    See Parsons et al. for details
    """

    # Turn the bond angle into an angle in [pi/2, pi]
    angle_BCD = pi - angle_BCD 

    # Calculate position of D from spherical coordinate representation
    D2 = Vector(cos(angle_BCD), cos(torsion_BC)*sin(angle_BCD), sin(torsion_BC)*sin(angle_BCD))**bond_CD

    # Calculate rotation matrix M
    bc = (C-B) / float(bond_BC)  # Normalized by previous bond length
    n = (B-A)**bc
    n.normalize()                # Normalized by calculation
    nXbc = n**bc                 # Normalized by definition

    M = (array([bc.get_array(), nXbc.get_array(), n.get_array()])).T

    # Calculate position of D by rotation and translation
    D = D2.left_multiply(M) + C
    
    return D

def from3atoms(A,B,C,alpha,beta,l,choice):
    gamma = calc_angle(B,A,C)

    Dx = l*cos(alpha)
    Dy = l*(cos(beta)-cos(alpha)*cos(gamma))/sin(gamma)
    Dz = sqrt(l**2-Dx**2-Dy**2)

    Dab   = Dx-cos(gamma)/sin(gamma)*Dy
    Dac   = Dy/sin(gamma)
    Dabac = Dz

    AB    = (B-A).normalized()
    AC    = (C-A).normalized()
    ABxAC = (AB**AC).normalized()

    if choice == 0:
        AD = AB**Dab + AC**Dac + ABxAC**Dabac
    else:
        AD = AB**Dab + AC**Dac - ABxAC**Dabac

    return A+AD


def from4atoms(OA, OB, OC, OD, u, v, l, m, accept_no_solution=True):
    u = pi - u
    v = pi - v
    
    a = OA - OC
    b = OB - OD
    
    ddiff = OC - OD + a - b # (BA) = (B-A)
    x = a.norm() * l * cos(u)
    y = b.norm() * m * cos(v)

    rx = x
    ry = y - b * ddiff

    r1 = a
    r2 = b

    if(abs(r1[0]) < 0.000001):
        if(abs(r2[0]) < 0.000001):
            print("WARNING: r1 = r2 = 0.0")        
        # Swap rows
        tmp = r1
        r1 = r2
        r2 = tmp
        tmp = rx
        ry = rx
        rx = tmp

    # Reduce the matrix
    factor = r1[0]
    r1 = r1 / factor
    rx = rx / factor    
    factor = r2[0]
    r2 = r2 - (r1 ** factor)
    ry = ry - rx * factor
    factor = r2[1]
    r2 = r2 / factor
    ry = ry / factor
    factor = r1[1]
    r1 = r1 - r2 ** factor
    rx = rx - ry * factor

    # Make solution space vectors
    alpha = r1[2]
    beta  = r2[2]
    gamma = rx
    delta = ry
    u = Vector(-alpha,-beta,1.0)
    v = Vector(gamma, delta, 0)

    # Solve quadratic equation for norm of c
    acoef = (u.norm())**2
    bcoef = 2*(u*v)
    ccoef = (v.norm())**2 - l**2
    disc = bcoef**2 - 4*acoef*ccoef

    if(disc < 0.0):
        # This is the sick case where we can't find _any_ solution
        if not accept_no_solution:
            raise ValueError("from4atoms: no solution found (disc=%f)" % disc)
        else:
            disc = 0
    
    x1 = (-bcoef - sqrt(disc))/(2*acoef)
    x2 = (-bcoef + sqrt(disc))/(2*acoef)

    # Create the two c-vectors
    c1 = u ** x1 + v
    c2 = u ** x2 + v

    # The two candidate E-poitns
    E1 = OA + c1
    E2 = OA + c2

    # The two candidate d-vectors
    d1 = E1 - OB
    d2 = E2 - OB

    # Pick the one with smallest norm difference
    d1norm = d1.norm()
    d2norm = d2.norm()
    diff1 = abs(d1norm - m)
    diff2 = abs(d2norm - m)

    if(diff1 < diff2):
        return E1
    else:
        return E2


def place_base(N_B, K_B, C_B, N_Bp, K_Bp, C_Bp, transform_list):
    # Calculate the basis for B'' and basis change matrix from B' to B''
    Xpp_Bp = (K_Bp-N_Bp).normalized()
    Zpp_Bp = ((K_Bp-N_Bp)**(C_Bp-N_Bp)).normalized()
    Ypp_Bp = Zpp_Bp**Xpp_Bp

    Bp_to_Bpp = ((matrix([Xpp_Bp.get_array(), Ypp_Bp.get_array(), Zpp_Bp.get_array()])).T).I

    # Calculate the basis vectors of B'' in B and basis change matrix from B'' to B
    Xpp_B = (K_B-N_B).normalized()
    Zpp_B = ((K_B-N_B)**(C_B-N_B)).normalized()
    Ypp_B = Zpp_B**Xpp_B

    Bpp_to_B = (matrix([Xpp_B.get_array(), Ypp_B.get_array(), Zpp_B.get_array()])).T

    # Calculate the change matrix from B' to B
    Bp_to_B = (Bpp_to_B * Bp_to_Bpp).A
    
    # Do the transformation for the whole list
    transform_list = [(Vp-N_Bp).left_multiply(Bp_to_B)+N_B for Vp in transform_list]
    
    return transform_list




In [16]:
def deg2rad(deg):
    return deg * (pi/180)

def rad2deg(rad):
    return rad * (180/pi)


## Distances (all in amino acids)

# Distances in backbone
d_PO5p   = 1.593                  # std. 0.010
d_O5pC5p = 1.440                  # std. 0.016 
d_C5pC4p = 1.510                  # std. 0.013
d_C4pC3p = 1.524                  # std. 0.011
d_C3pO3p = 1.423                  # std. 0.014
d_O3pP   = 1.607                  # std. 0.012

d_PO1P   = 1.485                  # std. 0.017
d_PO2P   = 1.485                  # std. 0.017


# Distances in sugar
# General
d_C1pC2p = 1.528                  # std. 0.010
d_C2pC3p = 1.525                  # std. 0.011
d_C3pC4p = 1.524                  # std. 0.011
d_C4pO4p = 1.453                  # std. 0.012
d_O4pC1p = 1.414                  # std. 0.012
d_C2pO2p = 1.413                  # std. 0.013
d_C1pN   = 1.471                  # std. 0.017  


# C2 endo specific
d_2e_C1pC2p = 1.526                  # std. 0.008
d_2e_C2pC3p = 1.525                  # std. 0.011
d_2e_C3pC4p = 1.527                  # std. 0.011
d_2e_C4pO4p = 1.454                  # std. 0.010
d_2e_O4pC1p = 1.415                  # std. 0.012
d_2e_C3pO3p = 1.427                  # std. 0.012
d_2e_C5pC4p = 1.509                  # std. 0.012
d_2e_C2pO2p = 1.412                  # std. 0.013
d_2e_C1pN   = 1.464                  # std. 0.014
d_2e_O5pC5p = 1.424                  # std. 0.016


# C3 endo specific
d_3e_C1pC2p = 1.529                  # std. 0.011
d_3e_C2pC3p = 1.523                  # std. 0.011
d_3e_C3pC4p = 1.521                  # std. 0.010
d_3e_C4pO4p = 1.451                  # std. 0.013
d_3e_O4pC1p = 1.412                  # std. 0.013
d_3e_C3pO3p = 1.417                  # std. 0.014
d_3e_C5pC4p = 1.508                  # std. 0.007
d_3e_C2pO2p = 1.420                  # std. 0.010
d_3e_C1pN   = 1.483                  # std. 0.015
d_3e_O5pC5p = 1.420                  # std. 0.009


# Distances in bases



# In base Adenine
d_A_N1C2 = 1.339                  # std. 0.009
d_A_C2N3 = 1.331                  # std. 0.009
d_A_N3C4 = 1.344                  # std. 0.006
d_A_C4C5 = 1.383                  # std. 0.007
d_A_C5C6 = 1.406                  # std. 0.009
d_A_C6N1 = 1.351                  # std. 0.007
d_A_C5N7 = 1.388                  # std. 0.006
d_A_N7C8 = 1.311                  # std. 0.007
d_A_C8N9 = 1.373                  # std. 0.008
d_A_N9C4 = 1.374209591    # 1.374 # std. 0.006
d_A_C6N6 = 1.335                  # std. 0.008
d_A_N9C1p= 1.462                  # std. 0.010


# In base Cytosine
d_C_N1C2 = 1.3965761705   # 1.397 # std. 0.010
d_C_C2N3 = 1.353                  # std. 0.008
d_C_N3C4 = 1.335                  # std. 0.007
d_C_C4C5 = 1.425                  # std. 0.008
d_C_C5C6 = 1.339                  # std. 0.008
d_C_C6N1 = 1.367                  # std. 0.006
d_C_C2O2 = 1.240                  # std. 0.009
d_C_C4N4 = 1.335                  # std. 0.009
d_C_N1C1p= 1.470                  # std. 0.012


# In base Guanine
d_G_N1C2 = 1.373                  # std. 0.008
d_G_C2N3 = 1.323                  # std. 0.008
d_G_N3C4 = 1.350                  # std. 0.007
d_G_C4C5 = 1.379                  # std. 0.007
d_G_C5C6 = 1.419                  # std. 0.010
d_G_C6N1 = 1.391                  # std. 0.007
d_G_C5N7 = 1.388                  # std. 0.006
d_G_N7C8 = 1.305                  # std. 0.006
d_G_C8N9 = 1.374                  # std. 0.007
d_G_N9C4 = 1.375420951    # 1.375 # std. 0.008
d_G_C2N2 = 1.341                  # std. 0.010
d_G_C6O6 = 1.237                  # std. 0.009
d_G_N9C1p = 1.459                 # std. 0.009


# In base Uracil
d_U_N1C2 = 1.38052345145  # 1.381 # std. 0.009
s_U_C2N3 = 1.373                  # std. 0.007
d_U_N3C4 = 1.380                  # std. 0.009
d_U_C4C5 = 1.431                  # std. 0.009
d_U_C5C6 = 1.337                  # std. 0.009
d_U_C6N1 = 1.375                  # std. 0.009
d_U_C2O2 = 1.219                  # std. 0.009
d_U_C4O4 = 1.232                  # std. 0.008
d_U_N1C1p = 1.469                 # std. 0.014



## Bond angles (all in radians)


# In backbone
a_O3pPO5p   = deg2rad( 104.0 )    # std. 1.9
a_PO5pC5p   = deg2rad( 120.9 )    # std. 1.6
a_C5pC4pC3p = deg2rad( 115.5 )    # std. 1.5
a_C4pC3pO3p = deg2rad( 110.6 )    # std. 2.6
a_C3pO3pP   = deg2rad( 119.7 )    # std. 1.2

a_O5pPO1P   = deg2rad( 108.1 )    # std. 2.9
a_O5pPO2P   = deg2rad( 108.3 )    # std. 2.7
a_O3pPO1P   = deg2rad( 107.4 )    # std. 3.2
a_O3pPO2P   = deg2rad( 108.3 )    # std. 3.2


# In sugar
a_C1pC2pC3p = deg2rad( 101.5 )    # std. 0.9
a_C2pC3pC4p = deg2rad( 102.7 )    # std. 1.0
a_C3pC4pO4p = deg2rad( 105.5 )    # std. 1.4
a_C4pO4pC1p = deg2rad( 109.6 )    # std. 0.9
a_O4pC1pC2p = deg2rad( 106.4 )    # std. 1.4
a_C1pC2pO2p = deg2rad( 110.6 )    # std. 3.0
a_C3pC2pO2p = deg2rad( 113.3 )    # std. 2.9
a_C2pC3pO3p = deg2rad( 111.0 )    # std. 2.8
a_C5pC4pO4p = deg2rad( 109.2 )    # std. 1.4
a_O4pC1pN   = deg2rad( 108.2 )    # std. 1.0
a_C2pC1pN   = deg2rad( 113.4 )    # std. 1.6
a_O5pC5pC4p = deg2rad( 111.6 )    # std. 1.7
a_C1pN9C4   = deg2rad( 127.1 )    # std. 1.8
a_C1pN1C2   = deg2rad( 117.9 )    # std. 1.3


# C2 endo specific
a_2e_C1pC2pC3p = deg2rad( 101.5 )    # std. 0.8
a_2e_C2pC3pC4p = deg2rad( 102.6 )    # std. 1.0
a_2e_C3pC4pO4p = deg2rad( 106.1 )    # std. 0.8
a_2e_C4pO4pC1p = deg2rad( 109.7 )    # std. 0.7
a_2e_O4pC1pC2p = deg2rad( 105.8 )    # std. 1.0
a_2e_C1pC2pO2p = deg2rad( 111.8 )    # std. 2.6
a_2e_C3pC2pO2p = deg2rad( 114.6 )    # std. 2.2
a_2e_C2pC3pO3p = deg2rad( 109.5 )    # std. 2.2
a_2e_C4pC3pO3p = deg2rad( 109.4 )    # std. 2.1
a_2e_C5pC4pC3p = deg2rad( 115.2 )    # std. 1.4
a_2e_C5pC4pO4p = deg2rad( 109.1 )    # std. 1.2
a_2e_O4pC1pN   = deg2rad( 108.2 )    # std. 0.8
a_2e_C2pC1pN   = deg2rad( 114.0 )    # std. 1.3
a_2e_O5pC5pC4p = deg2rad( 111.7 )    # std. 1.9
a_2e_C1pN9C4   = deg2rad( 127.4 )    # std. 1.2
a_2e_C1pN1C2   = deg2rad( 118.5 )    # std. 1.1


# C3 endo specific
a_3e_C1pC2pC3p = deg2rad( 101.3 )    # std. 0.7
a_3e_C2pC3pC4p = deg2rad( 102.6 )    # std. 1.0
a_3e_C3pC4pO4p = deg2rad( 104.0 )    # std. 1.0
a_3e_C4pO4pC1p = deg2rad( 109.9 )    # std. 0.8
a_3e_O4pC1pC2p = deg2rad( 107.6 )    # std. 0.9
a_3e_C1pC2pO2p = deg2rad( 108.4 )    # std. 2.4
a_3e_C3pC2pO2p = deg2rad( 110.7 )    # std. 2.1
a_3e_C2pC3pO3p = deg2rad( 113.7 )    # std. 1.6
a_3e_C4pC3pO3p = deg2rad( 113.0 )    # std. 2.0
a_3e_C5pC4pC3p = deg2rad( 116.0 )    # std. 1.6
a_3e_C5pC4pO4p = deg2rad( 109.8 )    # std. 0.9
a_3e_O4pC1pN   = deg2rad( 108.5 )    # std. 0.7
a_3e_C2pC1pN   = deg2rad( 112.0 )    # std. 1.1
a_3e_O5pC5pC4p = deg2rad( 111.5 )    # std. 1.6
a_3e_C1pN9C4   = deg2rad( 126.3 )    # std. 2.8
a_3e_C1pN1C2   = deg2rad( 116.7 )    # std. 0.6


# In bases

# In base Adenine 
a_A_C6N1C2  = deg2rad( 118.6 )    # std. 0.6
a_A_N1C2N3  = deg2rad( 129.3 )    # std. 0.5
a_A_C2N3C4  = deg2rad( 110.6 )    # std. 0.5
a_A_N3C4C5  = deg2rad( 126.8 )    # std. 0.7
a_A_C4C5C6  = deg2rad( 117.0 )    # std. 0.5
a_A_C5C6N1  = deg2rad( 117.7 )    # std. 0.5
a_A_C4C5N7  = deg2rad( 110.7 )    # std. 0.5
a_A_C5N7C8  = deg2rad( 103.9 )    # std. 0.5
a_A_N7C8N9  = deg2rad( 113.8 )    # std. 0.5
a_A_C8N9C4  = deg2rad( 105.8 )    # std. 0.4
a_A_N9C4C5  = deg2rad( 105.8 )    # std. 0.4
a_A_N3C4N9  = deg2rad( 127.4 )    # std. 0.8
a_A_C6C5N7  = deg2rad( 132.3 )    # std. 0.7
a_A_N1C6N6  = deg2rad( 118.6 )    # std. 0.6
a_A_C5C6N6  = deg2rad( 123.7 )    # std. 0.8
a_A_C8N9C1p = deg2rad( 127.7 )    # std. 1.8
a_A_C4N9C1p = deg2rad( 126.3 )    # std. 1.8


# In base Cytosine
a_C_C6N1C2  = deg2rad( 120.3 )    # std. 0.4
a_C_N1C2N3  = deg2rad( 119.2 )    # std. 0.7
a_C_C2N3C4  = deg2rad( 119.9 )    # std. 0.5
a_C_N3C4C5  = deg2rad( 121.9 )    # std. 0.4
a_C_C4C5C6  = deg2rad( 117.4 )    # std. 0.5
a_C_C5C6N1  = deg2rad( 121.0 )    # std. 0.5
a_C_N1C2O2  = deg2rad( 118.9 )    # std. 0.6
a_C_N3C2O2  = deg2rad( 121.9 )    # std. 0.7
a_C_N3C4N4  = deg2rad( 118.0 )    # std. 0.7
a_C_C5C4N4  = deg2rad( 120.2 )    # std. 0.7
a_C_C6N1C1p = deg2rad( 120.8 )    # std. 1.2
a_C_C2N1C1p = deg2rad( 118.8 )    # std. 1.1


# In base Guanine
a_G_C6N1C2  = deg2rad( 125.1 )    # std. 0.6
a_G_N1C2N3  = deg2rad( 123.9 )    # std. 0.6
a_G_C2N3C4  = deg2rad( 111.9 )    # std. 0.5
a_G_N3C4C5  = deg2rad( 128.6 )    # std. 0.5
a_G_C4C5C6  = deg2rad( 118.8 )    # std. 0.6
a_G_C5C6N1  = deg2rad( 111.5 )    # std. 0.5
a_G_C4C5N7  = deg2rad( 110.8 )    # std. 0.4
a_G_C5N7C8  = deg2rad( 104.3 )    # std. 0.5
a_G_N7C8N9  = deg2rad( 113.1 )    # std. 0.5
a_G_C8N9C4  = deg2rad( 106.4 )    # std. 0.4
a_G_N9C4C5  = deg2rad( 105.4 )    # std. 0.4
a_G_N3C4N9  = deg2rad( 126.0 )    # std. 0.6
a_G_C6C5N7  = deg2rad( 130.4 )    # std. 0.6
a_G_N1C2N2  = deg2rad( 116.2 )    # std. 0.9
a_G_N3C2N2  = deg2rad( 119.9 )    # std. 0.7
a_G_N1C6O6  = deg2rad( 119.9 )    # std. 0.6
a_G_C5C6O6  = deg2rad( 128.6 )    # std. 0.6
a_G_C8N9C1p = deg2rad( 127.0 )    # std. 1.3
a_G_C4N9C1p = deg2rad( 126.5 )    # std. 1.3


# In base Uracil
a_U_C6N1C2  = deg2rad( 121.0 )    # std. 0.6
a_U_N1C2N3  = deg2rad( 114.9 )    # std. 0.6
a_U_C2N3C4  = deg2rad( 127.0 )    # std. 0.6
a_U_N3C4C5  = deg2rad( 114.6 )    # std. 0.6
a_U_C4C5C6  = deg2rad( 119.7 )    # std. 0.6
a_U_C5C6N1  = deg2rad( 122.7 )    # std. 0.5
a_U_N1C2O2  = deg2rad( 122.8 )    # std. 0.7
a_U_N3C2O2  = deg2rad( 122.2 )    # std. 0.7
a_U_N3C4O4  = deg2rad( 119.4 )    # std. 0.7
a_U_C5C4O4  = deg2rad( 125.9 )    # std. 0.6
a_U_C6N1C1p = deg2rad( 121.2 )    # std. 1.4
a_U_C2N1C1p = deg2rad( 117.7 )    # std. 1.2




## Reference frame coordinates for the bases
# 


# Adenine
A_C1p = Vector(-2.479, 5.346, 0.000)
A_N9  = Vector(-1.291, 4.498, 0.000)
A_C8  = Vector( 0.024, 4.897, 0.000)
A_N7  = Vector( 0.877, 3.902, 0.000)
A_C5  = Vector( 0.071, 2.771, 0.000)
A_C6  = Vector( 0.369, 1.398, 0.000)
A_N6  = Vector( 1.611, 0.909, 0.000)
A_N1  = Vector(-0.668, 0.532, 0.000)
A_C2  = Vector(-1.912, 1.023, 0.000)
A_N3  = Vector(-2.320, 2.290, 0.000)
A_C4  = Vector(-1.267, 3.124, 0.000)

A_coords = [A_C1p,A_N9,A_C8,A_N7,A_C5,A_C6,A_N6,A_N1,A_C2,A_N3,A_C4]
A_names  = ["C1'","N9", "C8", "N7", "C5", "C6", "N6", "N1" , "C2", "N3", "C4"]
A_dict   = dict(list(zip(A_names, A_coords)))

# Cytosine
C_C1p = Vector(-2.477, 5.402, 0.000)
C_N1  = Vector(-1.285, 4.542, 0.000)
C_C2  = Vector(-1.472, 3.158, 0.000)
C_O2  = Vector(-2.628, 2.709, 0.000)
C_N3  = Vector(-0.391, 2.344, 0.000)
C_C4  = Vector( 0.837, 2.868, 0.000)
C_N4  = Vector( 1.875, 2.027, 0.000)
C_C5  = Vector( 1.056, 4.275, 0.000)
C_C6  = Vector(-0.023, 5.068, 0.000)

C_coords = [C_C1p,C_N1,C_C2,C_O2,C_N3,C_C4,C_N4,C_C5,C_C6]
C_names  = ["C1'", "N1", "C2", "O2", "N3", "C4", "N4", "C5", "C6"]
C_dict   = dict(list(zip(C_names, C_coords)))

# Guanine
G_C1p = Vector(-2.477, 5.399, 0.000)
G_N9  = Vector(-1.289, 4.551, 0.000)
G_C8  = Vector( 0.023, 4.962, 0.000)
G_N7  = Vector( 0.870, 3.969, 0.000)
G_C5  = Vector( 0.071, 2.833, 0.000)
G_C6  = Vector( 0.424, 1.460, 0.000)
G_O6  = Vector( 1.554, 0.955, 0.000)
G_N1  = Vector(-0.700, 0.641, 0.000)
G_C2  = Vector(-1.999, 1.087, 0.000)
G_N2  = Vector(-2.949, 0.139,-0.001)
G_N3  = Vector(-2.342, 2.364, 0.001)
G_C4  = Vector(-1.265, 3.177, 0.000)

G_coords = [G_C1p,G_N9,G_C8,G_N7,G_C5,G_C6,G_O6,G_N1,G_C2,G_N2,G_N3,G_C4]
G_names  = ["C1'","N9","C8","N7","C5","C6","O6","N1","C2","N2","N3","C4"]
G_dict   = dict(list(zip(G_names, G_coords)))

# Base Uracil
U_C1p = Vector(-2.481, 5.354, 0.000)
U_N1  = Vector(-1.284, 4.500, 0.000)
U_C2  = Vector(-1.462, 3.131, 0.000)
U_O2  = Vector(-2.563, 2.608, 0.000)
U_N3  = Vector(-0.302, 2.397, 0.000)
U_C4  = Vector( 0.989, 2.884, 0.000)
U_O4  = Vector( 1.935, 2.094,-0.001)
U_C5  = Vector( 1.089, 4.311, 0.000)
U_C6  = Vector(-0.024, 5.053, 0.000)

U_coords = [U_C1p,U_N1,U_C2,U_O2,U_N3,U_C4,U_O4,U_C5,U_C6]
U_names  = ["C1'","N1","C2","O2","N3","C4","O4","C5","C6"]
U_dict   = dict(list(zip(U_names, U_coords)))


bb_distances = (d_PO5p, d_O5pC5p, d_C5pC4p, d_C4pC3p, d_C3pO3p, d_O3pP)
bb_angles = (a_O3pPO5p, a_PO5pC5p, a_O5pC5pC4p, a_C5pC4pC3p, a_C4pC3pO3p, a_C3pO3pP)
bb_atomnames = ["P", "O5'", "C5'", "C4'", "C3'", "O3'"]

bb_distances_2e = (d_PO5p, d_O5pC5p, d_C5pC4p, d_2e_C3pC4p, d_C3pO3p, d_O3pP)
bb_angles_2e = (a_O3pPO5p, a_PO5pC5p, a_O5pC5pC4p, a_2e_C5pC4pC3p, a_2e_C4pC3pO3p, a_C3pO3pP)

bb_distances_3e = (d_PO5p, d_O5pC5p, d_C5pC4p, d_3e_C3pC4p, d_C3pO3p, d_O3pP)
bb_angles_3e = (a_O3pPO5p, a_PO5pC5p, a_O5pC5pC4p, a_3e_C5pC4pC3p, a_3e_C4pC3pO3p, a_C3pO3pP)





In [17]:
class BarnacleException(Exception): pass
class BarnacleBoundException(BarnacleException): pass
class BarnacleResampleException(BarnacleException): pass
class BarnacleStateException(BarnacleException): pass

In [18]:
    def sample_initial(self, start=None, end=None):
        """
        Sample new values for the dihedral angles.

        A subsequence of the angles can be sampled, conditioned on the
        remaining angles, if the parameters start and/or end are
        given.

        @param start: The first nucleotide position to sample angles
                      for (value None means 0).
        @type start: int

        @param end: The last nucleotide position to sample angles for
                    is end-1 (value None means len(nucleotide_sequence))
        @type end: int
        """
        
        self._save_prev()
        
        
        ###########################################################


        # Set default values of start and end
        if start == None:
            start = 0
        if end == None:
            end = len(self.nucs)

        # Check values of start and end
        if not 0<=start<end<=len(self.nucs):
            raise BarnacleBoundException("(start:end) out of bounds")
        if end-start != len(self.nucs) and not self.prev!=None:
            raise BarnacleResampleException("Cannot resample a subsequence, when no full sample has been drawn yet!")

        # Do the sampling and conversion
        self.inf_engine.set_start_end(start*num_angles, end*num_angles)

        while True:
            sample, slice_count = next(self.sample_generator)
            anglelist = data_to_anglelist(sample)
            try:

                residues = convert_from_dihedrals(anglelist, self.nucs, accept_no_solution=False)
            except ConversionException as e:
                self.inf_engine.undo()
                continue
            break

        # Save the sampled structure and angles
        self.data = sample
        self.anglelist = anglelist
        self.structure = residues_to_struct([('A', residues)], 'BARNACLE')
 

In [19]:
class Barnacle:
    """
    A probabilistic model of RNA conformational space.

    The class is initialized with a nucleotide sequence. Subsequent
    calls to sample, will sample RNA structures that are compatible
    with the given sequence and that have good local conformational
    geometry.

    >>> model = Barnacle('ACCU')
    >>> model.sample()
    >>> model.save_structure('barnacle01.pdb')
    >>> model.sample()
    >>> model.save_structure('barnacle02.pdb')
    >>>
    """

    def __init__(self, nucleotide_sequence):
        """
        Class constructor. The class must be initialized with a
        nucleotide sequence.

        @param nucleotide_sequence: the nucleotide sequence
        @type nucleotide_sequence: string
        """

        # Check that nucleotide_sequence only contains valid nucleotides
        for nuc in nucleotide_sequence:
            if not nuc in rna_one_letter_codes:
                raise BarnacleException("Nucleotide sequence contains non-RNA nucleotide '%s'" % nuc)


        # Set basic variables
        self.nucs = nucleotide_sequence

        # Set up the DBN
        self.dbn = make_model()

        # Setup interface for writing PDB-files
        self.pdbio = PDBIO()

        # Make data and mismask
        self.data, self.mism = make_data_and_mism(len(self.nucs))
        self.structure = None
        self.anglelist = None
        self.prev = None
        
        # Setup the sampler
        self.sampler = SamplerFwBt(self.dbn, hd_pos)
        self.inf_engine = InfEngineIOHMM(self.dbn, self.sampler, self.data, self.mism, support_undo=True)
        self.sample_generator = self.inf_engine.get_sample_generator()

        # Setup the likelihood calculator
        self.ll_calc = LikelohoodIOHMM_Fw(self.dbn, hd_pos)

    def _save_prev(self):
        self.prev = (self.data.copy(), self.structure, self.anglelist)

    def _restore_prev(self):
        assert(self.prev!=None)
        (self.data, self.structure, self.anglelist) = self.prev
        self.prev = None

    def sample(self, start=None, end=None, num = None):
        """
        Sample new values for the dihedral angles.

        A subsequence of the angles can be sampled, conditioned on the
        remaining angles, if the parameters start and/or end are
        given.

        @param start: The first nucleotide position to sample angles
                      for (value None means 0).
        @type start: int

        @param end: The last nucleotide position to sample angles for
                    is end-1 (value None means len(nucleotide_sequence))
        @type end: int
        """
        Energy = []
        # Save previous state
        print("Previous energy")
        if self.structure != None:
            df = coords_xyz_barn(self.structure)
            df = df[~top[4]]
            ## Calculate all distances between atoms
            dists=cdist(df.iloc[:,], df.iloc[:,], metric='euclidean')
            combs_close = top[0]
            myseq = list(range(0, len(combs_close), 1))
            energies = []
            for i in myseq:
                pair = top[1][i]
                ## Substract desired distances based on presaved indices
                distances=dists[top[2][i][0],:][:,top[2][i][1]].flatten()
                
                #Check if distances are lower than 1.5A or higher than 10A
                check_low  = np.any(distances < 1.5)
                #check_up = np.any(distances > 10)
            
                #Penalize if above condition is true
            
                if check_low == True:
                    energies.append(10000000000000000000)
                
                ## Calculate powers
                dd = potency2(distances).reshape(1, -1)
                modelname = str(pair + '_model.predict(dd)')
                #modelname = modelname[3:23]
                #print(modelname)
                out = round(eval(modelname)[0], 6)
                #print(top[0][i], pair, out)
                #print(pair)
                energies.append(out)
            out=round(sum(energies), 4)
            print(out)
            Energy.append(out)
        else:
            Energy.append(10000000000000000000000000000000000)
        
        self._save_prev()
        
        
        ###########################################################
        Energy.append(abs(Energy[0]*10000000000))
        #print("energy before while")
        #print(Energy)
        #print("......")
        

        # Set default values of start and end
        if start == None:
            start = 0
        if end == None:
            end = len(self.nucs)

        # Check values of start and end
        if not 0<=start<end<=len(self.nucs):
            raise BarnacleBoundException("(start:end) out of bounds")
        if end-start != len(self.nucs) and not self.prev!=None:
            raise BarnacleResampleException("Cannot resample a subsequence, when no full sample has been drawn yet!")
        
        if(Energy[0] == 10000000000000000000000000000000000):
            # Do the sampling and conversion
            self.inf_engine.set_start_end(start*num_angles, end*num_angles)

            while True:
                sample, slice_count = next(self.sample_generator)
                anglelist = data_to_anglelist(sample)
                try:

                    residues = convert_from_dihedrals(anglelist, self.nucs, accept_no_solution=False)
                except ConversionException as e:
                    self.inf_engine.undo()
                    continue
                break

            # Save the sampled structure and angles
            self.data = sample
            self.anglelist = anglelist
            self.structure = residues_to_struct([('A', residues)], 'BARNACLE')
            print("Finished generating the first sample ##############")
        else:
            while(Energy[0] < Energy[1]):

                print("Doing resampling")

                # Do the sampling and conversion
                self.inf_engine.set_start_end(start*num_angles, end*num_angles)

                while True:
                    sample, slice_count = next(self.sample_generator)
                    anglelist = data_to_anglelist(sample)
                    try:

                        residues = convert_from_dihedrals(anglelist, self.nucs, accept_no_solution=False)
                    except ConversionException as e:
                        self.inf_engine.undo()
                        continue
                    break

                # Save the sampled structure and angles
                self.data = sample
                self.anglelist = anglelist
                if(self.structure == None):
                    self.structure = residues_to_struct([('A', residues)], 'BARNACLE')
                    Energy.pop(1)
                    Energy.append(-99999999999999)
                    #print("Got here")
                    print(Energy)
                else:
                    self.structure = residues_to_struct([('A', residues)], 'BARNACLE')
                    df = coords_xyz_barn(self.structure)
                    print("New energy")
                    df = df[~top[4]]
                    ## Calculate all distances between atoms
                    dists=cdist(df.iloc[:,], df.iloc[:,], metric='euclidean')

                    combs_close = top[0]
                    myseq = list(range(0, len(combs_close), 1))

                    #Check if we have any distance with less of 1.5 in all the pairs
                    distance_tot =[]
                    for i in myseq:
                        distances=dists[top[2][i][0],:][:,top[2][i][1]].flatten()
                        distance_tot.append(distances)

                    #Convert to a single list
                    flat_list = []
                    for sublist in distance_tot:
                        for item in sublist:
                            flat_list.append(item)

                    flat_list = np.array(flat_list)
                    Minimums = any((flat_list < 1.5) == True)
                    print("THERE IS < 1.5 DISTANCES")
                    print(Minimums)

                    if(Minimums == True):
                        if(len(Energy) == 2):
                            Energy.pop(1)
                        out = 9000000000000000000000
                        Energy.append(out)
                    else:

                        energies = []
                        for i in myseq:
                            pair = top[1][i]
                            ## Substract desired distances based on presaved indices
                            distances=dists[top[2][i][0],:][:,top[2][i][1]].flatten()

                            #Check if distances are lower than 1.5A or higher than 10A
                            #check_low  = np.any(distances < 1.5)
                            #check_up = np.any(distances > 10)

                            #Penalize if above condition is true

                            #if check_low == True:
                            #    energies.append(10000000000000000000)


                            ## Calculate powers
                            dd = potency2(distances).reshape(1, -1)
                            modelname = str(pair + '_model.predict(dd)')
                            #modelname = modelname[3:23]
                            #print(modelname)
                            out = round(eval(modelname)[0], 6)
                            #print(top[0][i], pair, out)
                            #print(pair)
                            energies.append(out)
                        out=round(sum(energies), 4)

                        if(len(Energy) == 2):
                            Energy.pop(1)
                        Energy.append(out)
                        #print(out)

                        #print('Energy array is')
                        print(Energy)
                        #print(len(Energy))

                        #if len(Energy) == 2:
                            #print(Energy)
                        #    if (Energy[0] >= Energy[1]):
                        #        self._restore_prev()
                            #else:
                                #self.save_structure_pdb("structure_%03d.pdb" % num)
                        #print("HERE")
                        #if (Energy[1] > Energy[0]):
                        #        self._restore_prev()
                        #        print("Came to the previous conf")


                #In case there are not clashes
                #print("Printing energy 0")
                #print(Energy[0])
                #print(Energy[1] != 9000000000000000000000)
                if(Energy[1] != 9000000000000000000000):

                    #Apply random uniform condition to save structure even if energy is higher
                    random_distr = np.random.uniform(0,1,1)
                    print("The random distribution is")
                    print(random_distr)

                    if(random_distr > 0.2):
                        #print("Printing energies isnside the random_distr omg")
                        print(Energy)
                        if(Energy[0] > Energy[1]):
                            if(Energy[1] != -99999999999999):  
                                print("i am here")
                                self.save_structure_pdb("structure_%03d.pdb" % num)
                    else:
                        if(Energy[1] != -99999999999999):
                                self.save_structure_pdb("structure_%03d.pdb" % num)
                else:
                    print("Not saving the structure, there are clashes")
                    #print(Energy)
                    #print("##########")

                #else:
                #    self._restore_prev()
                #    print("Printing energies isnside the random_distr")
                #    print(Energy)
                #    if(Energy[0] > Energy[1]):
                #        if(Energy[1] != -99999999999999):  
                #            print("i am here")
                #            self.save_structure_pdb("structure_%03d.pdb" % num)
 
    def undo(self):
        """
        Undo the latest call to Barnacle.sample(), i.e. set the state
        of the class to what is was before sample was called. This
        method can only be called after Barnacle.sample() and only
        once for each call to Barnacle.sample().
        """
        if self.prev==None:
            raise BarnacleStateException("Barnacle.undo() is only allowed after a sample has been drawn. Multiple calls to undo are not supported.")

        if self.anglelist==None:
            raise BarnacleStateException("The first call to Barnacle.sample() cannot be followed by a call to Barnacle.undo().")

        self._restore_prev()
        self.inf_engine.undo()

    def get_log_likelihood(self):
        """
        Returns the log likelihood of the sampled angles (summed over all hidden states values).

        @return: the log likelihood of the sampled angles
        @rtype: float
        """
        if self.anglelist == None:
            raise BarnacleStateException("Barnacle.get_log_likelihood() can only be called after a sample has been drawn.")

        ll, slice_count = self.ll_calc.calc_ll(self.data, self.mism, set_data_in_dbn=False)
        return ll

    def get_angles(self):
        """
        Returns the sampled dihedral angles in the format. The angles
        are returned as a list of tuples of floats in the order:
        
        [(alpha, beta, gamma, delta, epsilon, zeta, chi), ..., (alpha, beta, gamma, delta, epsilon, zeta, chi)]

        Note that the angles are returned in radians.
        
        @return: the sampled angles
        @rtype: L{T{float}} object
        """

        return self.anglelist

    def get_structure(self):
        """
        Returns the atomic coordinates of the sampled structure as a
        Bio.PDB.Structure.
        
        @return: the sampled structure
        @rtype: Bio.PDB.Structure object
        """

        return self.structure

    def save_structure(self):
        """
        Save the atomic coordinates of the sampled structure as a PDB file.

        @param filename: The filename of the PDB file to save.
        @type filename: string
        """

        pdbio = PDBIO()
        pdbio.set_structure(self.structure)
        #pdbio.save(filename)
        
    def save_structure_pdb(self, filename="barnacle.pdb"):
        """
        Save the atomic coordinates of the sampled structure as a PDB file.

        @param filename: The filename of the PDB file to save.
        @type filename: string
        """

        pdbio = PDBIO()
        pdbio.set_structure(self.structure)
        pdbio.save(filename)

In [20]:
class AbstractInfEngine:
    """
    Abstract base class for inference engines.
    """

    def __init__(self, dbn, seq, weight):
        """
        @param dbn: DBN definition
        @type dbn: L{DBN} object

        @param seq: sequence data
        @type seq: numpy array

        @param weight: sequence weight
        @type weight: float
        """
        self.dbn=dbn
        self.seq=seq
        self.weight=weight
        self.seq_len=seq.shape[0]
        self.nodes_0, self.nodes_1=dbn.get_nodes()
        self.output_size=dbn.total_output_size

    def __len__(self):
        """
        Returns sequence length.

        @return: sequence length
        @rtype: int
        """
        return self.seq_len

    def _make_parentmap_list(self, seq, weight):
        """
        Construct a list of L{ParentMap} objects from the 
        sequence. 

        @param seq: sequence
        @type seq: numpy array

        @param weight: sequence weight
        @type weight: float
        
        @return: parent map for each node
        @rtype: list of L{ParentMap} objects
        """
        l=[]
        for node in self.dbn.unique_nodes:
            parentmap=node.get_parentmap(seq, weight)
            l.append(parentmap)
        return l

    def _set_parentmap(self, parentmap_list):
        """
        Put previously calculated L{ParentMap} objects (ie. for fast node value
        lookup in sequences) into the nodes.

        @param parentmap_list: list of parent maps
        @type parentmap_list: list of L{ParentMap} objects
        """
        for i in range(0, len(self.dbn.unique_nodes)):
            node=self.dbn.unique_nodes[i]
            parentmap=parentmap_list[i]
            assert(len(parentmap)==self.seq_len)
            node.set_parentmap(parentmap)

    def get_viterbi(self):
        """
        Calculate the Viterbi path. This method can act on a set of samples 
        (resulting in a stochastic Viterbi path calculation) or on all possible
        hidden node values for each slice (resulting in a deterministic Viterbi
        path calculation). The former is done by L{InfEngineMCMC}, the latter 
        by L{InfEngineHMM}.

        @return: (viterbi path, Loglik) tuple
        @rtype: (numpy array, float)
        """
        dbn=self.dbn
        seq=self.seq
        nr_slices=self.nr_slices
        seq_len=self.seq_len
        nodes_0=self.nodes_0
        nodes_1=self.nodes_1
        slices=self.slices
        core_seq=zeros(seq.shape, 'd')
        parentmap_list=self._make_parentmap_list(core_seq, self.weight)
        self._set_parentmap(parentmap_list)
        viterbi=zeros((seq_len, self.output_size), 'd')
        gain=zeros((seq_len, nr_slices), 'd')
        path=zeros((seq_len-1, nr_slices), 'i')
        # Initialization
        # l=0
        for i in range(0, nr_slices):
            slice=slices[0,i]
            core_seq[0]=slice
            ll=0.0
            for node in nodes_0:
                slice_log_lik, fv=node.get_slice_log_likelihood(0)
                ll+=slice_log_lik
            gain[0,i]=ll
        # Propagation
        # l>0
        for l in range(1, seq_len):
            for j in range(0, nr_slices):
                slice=slices[l,j]
                core_seq[l]=slice
                m=zeros(nr_slices, 'd')
                for i in range(0, nr_slices):
                    core_seq[l-1]=slices[l-1,i]
                    ll=0
                    for node in nodes_1:
                        slice_log_lik, fv=node.get_slice_log_likelihood(l)
                        ll+=slice_log_lik
                    m[i]=gain[l-1,i]+ll
                index=argmax(m)
                gain[l,j]=m[index]
                path[l-1,j]=index
        # Termination
        index=argmax(gain[seq_len-1,:])
        viterbi[seq_len-1,:]=slices[seq_len-1, index]
        # Backtracking
        for l in range(seq_len-2,-1,-1):
            # Here's some Numpy weirdness! 
            # path[a,b] returns an array of size (1,)!!!
            # This is so if it is of type int, not if type float!
            index=path[l,index][0]
            viterbi[l,:]=slices[l, index]
        # REMARK: remember that dbn.calc_ll puts the Viterbi sequence 
        # IN THE NODES so we make a copy of the Viterbi path to avoid 
        # side effects
        #ll=dbn.calc_ll(array(viterbi))
        #print "M=",max(gain[seq_len-1,:])/seq_len
        ll=max(gain[seq_len-1])/seq_len
        return viterbi, ll


if __name__=="__main__":

    pass




In [21]:
class InfEngineIOHMM(AbstractInfEngine):
    """
    Inference engines for inference in IO HMMs.
    """

    def __init__(self, dbn, sampler, seq, mismask, weight=1, support_undo=False):
        AbstractInfEngine.__init__(self, dbn, seq, weight)
        self.sampler=sampler
        self.mismask=mismask
        self.start=0
        self.end=len(seq)
        self.seq=seq

        self.support_undo=support_undo
        if support_undo:
            self.prev_seq=array(seq)
        else:
            self.prev_seq=None

        if weight!=1:
            raise MocapyException("Weight different from 1 not supported")

        # List of Family objects - one for each node
        # The Family objects tie the data and the nodes together
        self.parentmap_list=self._make_parentmap_list(seq, weight)

    def get_sample_generator(self, mismask=None):
        """
        Return a sample generator (which is a Python generator object).
        The sample generator's 'next' method will return a sampled sequence. 
        """
        if mismask is None:
            mismask=self.mismask
        sampler=self.sampler

        # Making sure we start with the original node values
        self._set_parentmap(self.parentmap_list) 

        # Start the real thing
        while 1:
            if self.support_undo:
                self.prev_seq=array(self.seq)
            slice_count=sampler.sweep(mismask=mismask, start=self.start, end=self.end)
            sampled_seq=array(self.seq)
            yield (sampled_seq, slice_count)


    def set_start_end(self, start, end):
        """
 
        @param start: start position in the sequence 
        @type start: int

        @param end: end position in the sequence
        @type end: int
        """
        if not 0<=start<self.seq_len or not start<end<=self.seq_len:
            raise MocapyException("Start:end out of bounds")
        self.start=start
        self.end=end

    def undo(self):
        """
        Undo last sampling step.
        """
        if self.prev_seq is None:
            raise MocapyException("Undo functionality was not enabled")
        for pm in self.parentmap_list:
            pm.replace_seq(self.prev_seq)
        self.seq = self.prev_seq
        self.prev_seq = None

    def replace_seq(self, seq):
        if self.support_undo:
            self.prev_seq = self.seq
        for pm in self.parentmap_list:
            pm.replace_seq(seq)
        self.seq = seq

    def get_viterbi(self):
        raise NotImplementedError





In [22]:

def random_sign():
    """
    Return random sign (ie. 1 or -1)

    @return: random sign
    @rtype: int
    """
    if random()<0.5:
        return -1
    else:
        return 1

def close(x, y, delta=1e-10):
    """
    Return 1 if x is close to y (within delta).
    If x and y are list-like, the test is done for each 
    element pair of x and y.

    @param x, y: test if x is close to y
    @type x,y: int, float, numpy array, list,...

    @param delta: closeness cutoff
    @type delta: float

    @return: 1 if x is close to y, 0 otherwise
    @rtype: int (0 or 1)
    """
    type_x=type(x)
    if type_x==FloatType or type_x==IntType:
        # Float or Int
        if abs(x-y)<delta:
            return 1
        else:
            return 0
    else:
        # Array, list,...
        for i in range(0, len(x)):
            if abs(x[i]-y[i])>delta:
                return 0
        return 1

def gen_refmat(p, q):
    """
    Return a (left multiplying) matrix that reflects p onto q.
    p and q can be of arbitrary dimension.

    @param p,q: vectors, same length
    @type p,q: numpy array (shape=n, with n>1)

    @return: matrix that reflects p onto q
    @rtype: numpy array (shape=(n,n))
    """
    q=q/norm(q)
    p=p/norm(p)
    pq=p-q
    npq=norm(pq)
    if npq<1e-5:
        n=p.shape[0]
        return eye(n)
    pq=pq/npq
    n=p.shape[0]
    pq.shape=(n,1)
    i=eye(n)
    ref=i-2*dot(pq, transpose(pq))
    return ref


def gen_rotmat(p,q):
    """
    Return a (left multiplying) matrix that rotates p onto q.
    p and q can be of arbitrary dimension.

    @param p,q: vectors, same length
    @type p,q: numpy array (shape=n, with n>1)

    @return: matrix that reflects p onto q
    @rtype: numpy array (shape=(n,n))
    """
    q=q/norm(q)
    p=p/norm(p)
    npq=norm(p-q)
    if npq<1e-5:
        n=p.shape[0]
        return eye(n)
    rot=dot(gen_refmat(q, -p), gen_refmat(p, -p))
    return rot


def argmax(a):
    """
    Return the index of a maximum in sequence a.
    This is an alternative implementation of argmax: 
    if there are equivalent maxima, it picks one at 
    random. Numeric's argmax always returns the index
    of the first maximum.

    @param a: array of numbers
    @type a: list

    @return: index of maximum of a
    @rtype: int
    """
    m=a[0]
    indices=[0]
    for i in range(1, len(a)):
        x=a[i]
        if x>m:
            # New maximum
            m=x
            indices=[i]
        elif x==m:
            # Same maximum: append index
            indices.append(i)
    # Pick a random index of a maximum
    return choice(indices)


def norm(a):
    """
    Return norm of array a. 
    Basically a shortcut to avoid creating a Vector object.

    @param a: vector
    @type a: numpy array (shape=n, with n>1)

    @return: vector norm
    @rtype: double
    """
    return sqrt(sum(a*a))





In [23]:
def mocapy_seed(a, b):
    """
    Initialize the random number generators that are used
    by Mocapy. These are numpy.random and Python's random 
    module.

    @param a: seed for numpy.random
    @type a: int

    @param b: seed for Python's random module
    @type b: int
    """
    numpy.random.seed(a)
    random.seed(b)


def load_dbn(fname):
    """
    Load a pickled DBN object.

    @param fname: filename of the pickled DBN object
    @type fname: string

    @return: the unpickled DBN object
    @rtype: L{DBN} object
    """
    fp=open(fname, 'r')
    dbn=pickle.load(fp)
    fp.close()
    return dbn


class DBN:
    """
    This class stores the Node objects and performs the necessary initialisations. 
    In addition, it checks wether the DBN definition makes sense. The class can also
    be used to sample a sequence from the DBN, and to calculate the loglik of a 
    sequence (when all node values are known).
    """
    def __init__(self, nodes_0, nodes_1, name="DBN"):
        """
        @param nodes_0: nodes in slice 0.
        @type nodes_0: list of L{Node} objects.

        @param nodes_1: nodes in slice 1.
        @type nodes_1: list of L{Node} objects.

        @param name: optional name of the L{DBN}.
        @type name: string
        """
        # Flag - used to check proper initialisation
        self.is_constructed=0
        # Nodes in slice 0
        self.nodes_0=nodes_0
        # Nodes in slices 1-T
        self.nodes_1=nodes_1
        if len(nodes_0)!=len(nodes_1):
            raise MocapyDBNException("Slices need to have the same sumber of nodes.")
        # Number of nodes/slice
        self.nr_nodes=len(self.nodes_0)
        self.name=name
        # Set the node indices
        data_index=0
        for n in range(0, self.nr_nodes):
            n0=nodes_0[n]
            n1=nodes_1[n]
            # Identifier of the node
            n0.set_node_index(n)
            n1.set_node_index(n)
            # Where is the node data found?
            n0.set_data_index(data_index)
            n1.set_data_index(data_index)
            assert(n0.output_size==n1.output_size)
            # Move to next free index in data
            data_index+=n1.output_size
        # Map node name to index
        self.index_map=self._make_index_map(nodes_0, nodes_1)
        self.total_output_size=data_index

    def _make_index_map(self, nodes_0, nodes_1):
        """
        Create a dicationary that maps the name of a node to its index.

        @param nodes_0: nodes in slice 0.
        @type nodes_0: list of L{Node} objects.

        @param nodes_1: nodes in slice 1.
        @type nodes_1: list of L{Node} objects.

        @rtype: dictionary that maps node names to (index, slice) tuples
        """
        index_map={}
        for i in range(0, len(nodes_0)):
            node=nodes_0[i]
            if node.name!="":
                index_map[node.name]=(i, 0)
            node=nodes_1[i]
            if node.name!="":
                index_map[node.name]=(i, 1)
        return index_map
    
    def _map_to_indices(self, parent_i, child_i):
        """
        Return node indices associated with the two given node names 'parent'
        and 'child'. If a name is an integer, it is returned as is.

        @param parent_i: parent index or name
        @type parent_i: string or int

        @param child_i: child index or name
        @type child_i: string or int

        @return: (parent index, child index)
        @rtype: (int, int)
        """
        if not type(parent_i)==IntType:
            if parent_i not in self.index_map:
                raise MocapyDBNException("Unknown node name: %s" % str(parent_i))
            else:
                parent_i, slice=self.index_map[parent_i]
        if not type(child_i)==IntType:
            if child_i not in self.index_map:
                raise MocapyDBNException("Unknown node name: %s" % str(child_i))
            else:
                child_i, slice=self.index_map[child_i]
        return parent_i, child_i
            
    # Special

    def __repr__(self):
        """
        @return: "<DBN I{name} #nodes=I{number of nodes}>".
        @rtype: string
        """
        s="<DBN %s #nodes=%i>" % (self.name, self.nr_nodes)
        return s

    # Public

    def get_nodes(self):
        """
        Return nodes in slice l=0, and in slice l>0
        as two node lists.

        @return: (node list at l=0, node list at l>0)
        @rtype: ([L{Node},...,L{Node}], [L{Node},...,L{Node}])
        """
        return self.nodes_0, self.nodes_1

    def get_node_by_name(self, name):
        """
        Return the node object that is associated with the given name.

        @param name: name of the node
        @type name: string

        @return: node associated with name
        @rtype: L{Node}
        """
        index, slice=self.index_map[name]
        if slice==0:
            return self.nodes_0[index]
        else:
            return self.nodes_1[index]

    def sample_sequence(self, length):
        """
        Return a sampled sequence sequence from the DBN
        with specified length.

        @param length: length of data sequence
        @type length: int

        @return: a sequence with the hidden nodes sampled
        @rtype: array, shape=(sequence length, output size)
        """
        assert(self.is_constructed)
        seq=zeros(([length, self.total_output_size]), 'd')
        for node in self.unique_nodes:
            parentmap=node.get_parentmap(seq, weight=1)
            node.set_parentmap(parentmap)
        ll=0
        for l in range(0, length):
            if l==0:
                # Use nodes in slice 0
                node_list=self.nodes_0
            else:
                # Use nodes in slices 1-T
                node_list=self.nodes_1
            for node in node_list:
                node.sample(l)
                slice_log_ll, family=node.get_slice_log_likelihood(l)
                ll+=slice_log_ll
        return seq, ll/length

    def add_inter(self, parent_i, child_i):
        """
        Add an edge between slices, from parent to child.

        @param parent_i: parent node index or name 
        @type parent_i: int or string

        @param child_i: child node index or name 
        @type child_i: int or string
        """
        assert(not self.is_constructed)
        # Map names to indices if necessary
        parent_i, child_i=self._map_to_indices(parent_i, child_i)
        # slice 0
        parent_0=self.nodes_0[parent_i]
        # slice 1
        parent_1=self.nodes_1[parent_i]
        child_1=self.nodes_1[child_i]
        # Add children to parents
        parent_0.add_inter_child(child_1)
        # Note that child_0 cannot have parents
        child_1.add_inter_parent(parent_0.data_index, parent_0.node_size)
        # Parents tied?
        if not parent_0 is parent_1:
            parent_1.add_inter_child(child_1)

    def add_intra(self, parent_i, child_i):
        """
        Add an edge inside a slice, from parent to child.

        @param parent_i: parent node index or name 
        @type parent_i: int or string

        @param child_i: child node index or name 
        @type child_i: int or string
        """
        assert(not self.is_constructed)
        # Map names to indices if necessary
        parent_i, child_i=self._map_to_indices(parent_i, child_i)
        # slice 0
        parent_0=self.nodes_0[parent_i]
        child_0=self.nodes_0[child_i]
        # slice 1
        parent_1=self.nodes_1[parent_i]
        child_1=self.nodes_1[child_i]
        # Add children to parents
        parent_0.add_intra_child(child_0)
        if not parent_0 is parent_1:
            parent_1.add_intra_child(child_1)
        # Add parents to children
        child_0.add_intra_parent(parent_0.data_index, parent_0.node_size)
        if not child_0 is child_1:
            child_1.add_intra_parent(parent_0.data_index, parent_0.node_size)

    def construct(self):
        """
        Initialize the DBN data structures based on 
        the added nodes and edges. After calling this
        method no edges can be added.
        """
        assert(not self.is_constructed)
        # construct nodes in slice 0
        self.unique_nodes=[]
        for n in range(0, self.nr_nodes):
            # Slice 0
            node_0=self.nodes_0[n]
            node_1=self.nodes_1[n]
            if node_0 is node_1:                 
                # Tied node
                node_0.construct(slice="TIED")
                self.unique_nodes.append(node_0)
            else:
                node_0.construct(slice="START")
                node_1.construct(slice="END")
                self.unique_nodes.append(node_0)
                self.unique_nodes.append(node_1)
        self.all_nodes=self.nodes_0+self.nodes_1
        self.is_constructed=1

    def calc_ll(self, seq):
        """
        Calculate the loglik for a sequence (with all node values known).

        @return: LogLik (normalized for number of slices)
        @rtype: float
        """
        ll=0
        seq_len=seq.shape[0]
        for node in self.unique_nodes:
            parentmap=node.get_parentmap(seq, weight=1)
            node.set_parentmap(parentmap)
        for l in range(0, seq_len):
            if l==0:
                nodes=self.nodes_0
            else:
                nodes=self.nodes_1
            for n in range(0, self.nr_nodes):
                node=nodes[n]
                node_log_ll, fv=node.get_slice_log_likelihood(l)
                ll+=node_log_ll
        return ll/seq_len

    def get_parameter_count(self):
        """
        Return total number of parameters.

        @return: total number of parameters
        @rtype: int
        """
        pc=0
        for node in self.unique_nodes:
            pc+=node.par_count
        return pc

    def save(self, fname):
        """
        Save a persistent copy of the DBN object (ie. a pickled object).
        This persistent copy can be loaded again with 'load_dbn'.

        @param fname: filename
        @type fname: string
        """
        fp=open(fname, 'w')
        pickle.dump(self, fp)
        fp.close()






In [24]:
# This avoids log underflow
_MIN_TRANSITION=1e-50
_LOG_MIN_TRANSITION=log(_MIN_TRANSITION)


def normalize_cpd(cpd):
    """
    Normalize a CPD matrix (ie. so the sum of its values
    along the last axis is one).

    @param cpd: conditional probability distribution of a discrete node
    @type cpd: numpy array

    @return: normalized CPD
    @rtype: numpy array
    """
    s=sum(cpd+_MIN_TRANSITION, -1)
    if len(s.shape)==0:
        # CPD is a 0D array - sum is a float
        return cpd/s
    else:
        # CPD is an ND array - sum is an array
        return cpd/s[..., newaxis]

def make_random_cpd(sizes, no_zeros=1):
    """
    Return a random CPD.

    @param sizes: size of the CPD, the shape of a numpy array
    @type sizes: tuple of int

    @param no_zeros: if 1, make sure the CPD has no 0 entries
    @type no_zeros: int (0 or 1)

    @return: random CPD
    @rtype: numpy array
    """
    cpd=random(sizes)
    # avoid zero entries
    if no_zeros:
        cpd=clip(cpd, 0.1, 2)
    cpd=normalize_cpd(cpd)
    return cpd


class DiscreteNode(Node):
    """
    Discrete node.
    """
    def __init__(self, node_size, user_cpd=None, prior=None, name=""):
        """
        @param node_size: number of values the node can adopt
        @type node_size: int

        @param user_cpd: optional user supplied CPD table
        @type user_cpd: numpy array

        @param prior: a prior object
        @type prior: object from L{DiscretePriors}
        """
        # Number of values the node can adopt
        self.node_size=node_size
        # Use custom CPD if present
        self.user_cpd=user_cpd
        self.arange_node_size=arange(0, node_size)
        self.prior=prior
        self.name=name
        self.counts=None
        # Parameter count
        Node.__init__(self, output_size=1, node_type="DISCRETE")

    # Special

    def __repr__(self):
        return "<DiscreteNode name=%s size=%i\n%s\n>" % \
                (self.name, self.node_size, self.par2str())

    # Private

    def _test_cpd(self, cpd, cpd_shape):
        """
        Check if user supplied CPD table has the right shape.
        If the shape is wrong, a L{MocapyDiscreteException} is raised.

        @param cpd: CPD table
        @type cpd: numpy array

        @param cpd_shape: CPD table shape
        @type cpd_shape: tuple of ints
        """
        if len(cpd_shape)!=len(cpd.shape):
            raise MocapyDiscreteException("User CPD has wrong dimensions.")
        for i in range(0, len(cpd_shape)):
            dim1=cpd_shape[i]
            dim2=cpd.shape[i]
            if dim1!=dim2:
                raise MocapyDiscreteException("User CPD has wrong dimensions.")

    # Public

    def get_ess_list(self):
        return self.ess_list

    def get_parentmap(self, seq, weight=1):
        return ParentMap(seq, self.parents_0, self.parents_1, 
                         self.data_index, weight)

    def construct(self, slice):
        self.cpd_shape=(self.parents_0_sizes+self.parents_1_sizes+[self.node_size])
        # Parameter count
        self.par_count=multiply.reduce(self.cpd_shape)
        if not self.user_cpd is None:
            # Use user-provided CPD
            self.set_cpd(self.user_cpd)
            del self.user_cpd
        else:
            # Use random CPD
            cpd=make_random_cpd(self.cpd_shape)
            self.set_cpd(cpd)
        # Get ready for E-step
        self.ess=zeros(self.cpd_shape, 'd')
        self.ess_list=[zeros(self.cpd_shape, 'd')]
        Node.construct(self, slice)

    def set_cpd(self, new_cpd=None):
        if rank==0:
            assert(not new_cpd is None)
            cpd=normalize_cpd(new_cpd)
            # Get rid of very small values that lead to log overflow
            cpd=clip(cpd, _MIN_TRANSITION, 1000) 
            # Calculate log(cpd) for speed reasons
            print(type(cpd))
            log_cpd=log(cpd)
            cum_cpd=cumsum(cpd, -1)
            # Avoid bisect errors
            cum_cpd[...,-1]=2.0
            self.cpd, self.log_cpd, self.cum_cpd=bcast((cpd, log_cpd, cum_cpd))
            if __debug__:
                self._test_cpd(self.cpd, self.cpd_shape)
        else:
            self.cpd, self.log_cpd, self.cum_cpd=bcast()

    def sample(self, l):
        # Cast to tuple to avoid fancy indexing in numpy
        pv=tuple(self.parentmap[l][:-1])
        if len(pv)>0:
            cum_cpd=self.cum_cpd[pv]
        else:
            cum_cpd=self.cum_cpd
        #choice=very_simple_sampler(self.arange_node_size, cum_cpd)
        r=random()
        choice=bisect(cum_cpd, r)
        self.parentmap[l]=choice

    def update_ess(self, par_this_values):
        self.ess[par_this_values]+=1

    def save_ess(self):
        # Take sequence weight into account!
        self.ess_list[0]=self.ess_list[0]+self.weight*self.ess
        # Get ready for new ESS
        self.ess=zeros(self.cpd_shape, 'd')

    def blanket_sample(self, l):
        # local copies
        seq_len=self.seq_len
        cpd=self.cpd
        log_cpd=self.log_cpd
        children_1=self.children_1
        children_2=self.children_2
        parentmap=self.parentmap
        # Get parent+this values
        ptv=self.parentmap[l]
        # Probabilities
        p=zeros(self.node_size, 'd')
        # Loop over all proposed new states
        for state in range(0, self.node_size):
            # Set value of node in slice_1 to state
            ptv[-1]=state
            parentmap[l]=state
            # Not that we are working in LOG space!
            p[state]=log_cpd[tuple(ptv)]
            log_lik_product=0
            # Children in current slice
            for child in children_1:
                lik, cptv=child.get_slice_log_likelihood(l)
                log_lik_product+=lik
            # Children in next slice
            if l<(seq_len-1):
                # Skip this when we reach end of sequence  
                # (because there are no children in the next slice)
                for child in children_2:
                    lik, cptv=child.get_slice_log_likelihood(l+1)
                    log_lik_product+=lik
            p[state]+=log_lik_product
        # Get rid of potentially very large values in p
        p=p-max(p)
        # Get rid of potentially very small values in p
        p=clip(p, _LOG_MIN_TRANSITION, 1000)
        # Get out of LOG space
        p=exp(p)
        # Normalize probabilities
        cpd=p/sum(p)
        # Make cumulative cpd 
        cum_cpd=cumsum(cpd, -1)
        # Make sure cum_cpd sums to 1
        cum_cpd=cum_cpd/cum_cpd[-1]
        # Avoid problem with bisect
        cum_cpd[-1]=2
        # Choose a node value
        r=random()
        choice=bisect(cum_cpd, r)
        parentmap[l]=choice

    def get_slice_log_likelihood(self, l):
        ptv=tuple(self.parentmap[l])
        p=self.log_cpd[ptv]
        return p, ptv 

    def do_M_step(self, ess_list):
        if rank==0:
            # sum ESS
            ess=ess_list.pop()
            for a in ess_list:
                ess+=a
            # Save the real counts
            self.counts=array(ess)
            if self.prior:
                ess=self.prior.apply_prior(ess)
            self.set_cpd(ess)   
        else:
            self.set_cpd()
        # Get ready for new E-step
        self.ess_list=[zeros(self.cpd_shape, 'd')]

    def get_counts(self):
        return self.counts

    def get_parameters(self):
        """
        Return CPD table.

        @return: the node's CPD table
        @rtype: numpy array
        """
        return self.cpd

    def par2str(self, precision=2):
        s=array2string(self.cpd, precision=precision, max_line_width=200, 
                       suppress_small=1)
        return s




In [25]:
rank=0
size=1

SUM=0


def bcast(t):
    "Returns t"
    return t

def barrier():
    "Returns immediately"
    return

def reduce(x, operation):
    "Returns x"
    return x

def gather(l):
    "Returns l"
    return l

def scatter(l):
    "Returns l"
    return l


SUM = None




In [26]:
class MocapyException(Exception): pass

class MocapyVectorException(MocapyException): pass

class MocapyDBNException(MocapyException): pass

class MocapyVMFException(MocapyException): pass

class MocapyGaussianException(MocapyException): pass

class MocapyDiscreteException(MocapyException): pass

class MocapyDirichletException(MocapyException): pass

class MocapyKentException(MocapyException): pass

class MocapyEMException(MocapyException): pass

class MocapyVMException(MocapyException): pass

In [27]:

class MultiGauss:
    """
    Density function for multidimensional Gaussian distribution.
    """
    def __init__(self, mean, sigma):
        """
        @param mean: mean of Gaussian density 
        @type mean: float or array of type 'd'

        @param sigma: sigma of Gaussian density
        @type sigma: float or array of type 'd'
        """
        self.sigma=sigma
        self.mean=mean
        if type(mean)==FloatType and type(sigma)==FloatType:
            # 1D Gaussian
            self.c=-log(sigma*sqrt(2*pi))
            self.c2=2*sigma*sigma
            self.one_dim=1
        else:
            # Multidimensional Gaussian
            assert(det(sigma)>0)
            self.dim=mean.shape[0]
            assert(sigma.shape==(self.dim,self.dim))
            sq_det=sqrt(det(self.sigma))
            self.b=1.0/(power(2*pi, self.dim/2.0)*sq_det)
            self.logb=log(self.b)
            self.inv_sigma=inv(self.sigma)
            self.one_dim=0

    def __call__(self, a, log_space=0):
        """
        Return likelihood of a.

        @param a: argument to Gaussian density function
        @type a: float or array of type 'd'

        @param log_space: if 1 return log(density)
        @type log_space: 0 or 1

        @return: Gaussian density at a
        @rtype: float
        """
        if self.one_dim:
            # 1D Gaussian
            d=(a-self.mean)
            x=self.c-d*d/self.c2
            if log_space:
                return x
            else:
                return exp(x)
        else:
            # Multidimensional Gaussian
            p=a-self.mean
            q=dot(self.inv_sigma, p)
            r=-0.5*dot(transpose(p), q)
            if log_space:
                return self.logb+r
            else:
                return self.b*exp(r)

    def __repr__(self):
        return "<%d-dim Gaussian, mean=%s>" % (self.dim, str(self.mean))



In [28]:
class ParentMap:
    """
    L{ParentMap} keeps track of where the data describing the values of a specific 
    node is found in the data sequence.
    """
    def __init__(self, seq, parents_0, parents_1, this_index, weight):
        """
        @param seq: the data sequence
        @type seq: numpy array

        @param parents_0: index of parent value in previous slice 
        5B
        @type parents_0: int

        @param parents_1: index of parent value in current slice 
        @type parents_1: int

        @param this_index: index of node value itself in current slice
        @type this_index: int

        @param weight: sequence weight
        @type weight: float in [0,1]
        """
        # data
        lng=len(seq)
        self.lng=lng
        self.seq=seq.flat
        # self index
        # Cache the indices
        indices=[]
        # Step size
        if len(seq.shape)==1:
            step=1
        else:
           step=multiply.reduce(seq.shape[1:])
        for l in range(0, lng):
            index=[]
            if l>0:
                for pi in parents_0:
                    index.append((l-1)*step+pi)
            for pi in parents_1:
                index.append(l*step+pi)
            index.append(l*step+this_index)
            indices.append(index)
        self.this_index=this_index
        self.step=step
        self.indices=indices
        self.weight=weight

    def __setitem__(self, l, v):
        """
        Set data value of a node at position l.

        @param l: sequence index
        @type l: int

        @param v: data value
        @type v: array of type 'd'
        """
        index=self.indices[l][-1]
        self.seq[index]=v

    def __getitem__(self, l):
        """
        Return the parent/node values at position 'l'.

        @param l: sequence position
        @type l: int

        @return: parent/node values at position 'l' 
        @rtype: array, consisting of [parent value 1, parent value 2,..., node value]
        """
        index=self.indices[l]
        ptv=take(self.seq, index).astype('i')
        return ptv

    def __getstate__(self):
        """
        Dummy method to avoid pickling data.
        """
        return {}

    def __len__(self):
        return self.lng

    def get_weight(self):
        """
        Return the sequence weight.

        @return: sequence weight
        @rtype: float in [0,1]
        """
        return self.weight

    def replace_seq(self, seq):
        """ 
        Replace the sequence - used in undo operations.
        """
        if not len(seq)==self.lng:
            raise MocapyException("Replacing sequence has wrong shape")
        self.seq=seq.flat


class ContinuousParentMap(ParentMap):
    def __init__(self, seq, parent, this_index, dim, weight):
        self.dim=dim
        ParentMap.__init__(self, seq, [], [parent], this_index, weight)

    def __getitem__(self, l):
        i,j=self.indices[l]
        parent_value=int(self.seq[i])
        node_value=self.seq[j:j+self.dim]
        return (parent_value, node_value)

    def __setitem__(self, l, v):
        index=self.indices[l][-1]
        self.seq[index:index+self.dim]=v

In [29]:
class VMNode(Node):
    "Von Mises node for observations on a 2-dimensional sphere"
    "described by a direction in the interval from 0 to 2pi"
    def __init__(self, node_size, user_mus=None, user_kappas=None, name=""):
        """
        @param node_size: number of mixture components 
        @type node_size: int
        
        @param user_mus: user supplied mean directions
        @type user_mus: numpy array, shape=nsize x 1 

        @param user_kappas: user supplied Kappa parameters
        @type user_kappas: numpy array, shape=nsize
        """
        # Set different variable values
        self.node_size=node_size
        self.user_mus=user_mus
        self.user_kappas=user_kappas
        self.ess_r, self.ess_w=self._get_empty_ess()
        self.ess_list=[self._get_empty_ess()]
        self.name=name
        # Parameter count
        self.par_count=node_size*3
        # Initialize Node base class
        Node.__init__(self, output_size=1, node_type="VM")

    def __repr__(self):
        s="<VMNode name=%s size=%i\n%s\n>" % (self.name, self.node_size, self.par2str())
        return s

    def _get_empty_ess(self):
        """
        Return initialized ESS data structures.
        """
        ess_r=zeros((self.node_size, 2), 'd')
        ess_w=zeros(self.node_size, 'd')
        return [ess_r, ess_w]

    def _make_rnd_kms(self, node_size):
        """
        Make random Mu's and Kappa's.
        """
        mus=zeros((node_size,1), 'd')
        for i in range(0, node_size):
            mus[i]=uniform(0,2*pi)
        kappas=random(node_size)            
        return mus, kappas

    def _make_vm_list(self, mus, kappas, node_size):
        """
        Initialize the list of L{VMDens} objects.
        These objects calculate P(obs|mu, kappa), and
        are here pre-constructed for speed.
        """
        vm_list=[]
        samplers=[]
        for i in range(0, node_size):
            m=mus[i]
            k=kappas[i]
            vm=VMDens(k, m)
            vm_list.append(vm)
            sampler=VMSampler(k, m)
            samplers.append(sampler)
        return vm_list, samplers

    # Public
    def get_ess_list(self):
        return self.ess_list

    def construct(self, slice):
        Node.construct(self, slice)
        self.parent_index=self.parents_1[0]
        assert(len(self.parents_0)==0)
        assert(len(self.parents_1)==1)
        parent_size=self.parents_1_sizes[0]
        assert(self.node_size==parent_size)
        self.mus_shape=(self.node_size,1)
        self.kappas_shape=(self.node_size),
        # Check or construct means
        rnd_mus, rnd_kappas=self._make_rnd_kms(self.node_size)

        # If the user has specified some values these are used        
        if not self.user_mus is None:
            self.mus=self.user_mus
            for i in range(0, self.node_size):
                # Make sure we have angles in [0,2pi[
                self.mus[i]=mod2pi(self.mus[i])
            assert(self.mus.shape==self.mus_shape)
            del self.user_mus
        else:
            # Otherwise the means are set to random values
            self.mus=rnd_mus

        # Check or construct kappa values
        if not self.user_kappas is None:
            self.kappas=self.user_kappas
            assert(self.kappas.shape==self.kappas_shape)
            del self.user_kappas
        else:
            self.kappas=rnd_kappas
        self.vm_list, self.samplers=self._make_vm_list(self.mus, self.kappas, self.node_size)

    def get_parentmap(self, seq, weight):
        return ContinuousParentMap(seq, self.parent_index, self.data_index,
                                   1, weight)

    def update_ess(self, family_values):
        # Get which node value the parent has
        parent_value=family_values[0]
        node_value=family_values[1]
        # Add ess info on the current node value to the current info on
        # the mixture element corresponding to the parent value
        self.ess_r[parent_value]+=array([cos(node_value), sin(node_value)])
        self.ess_w[parent_value]+=1

    def save_ess(self):
        if self.weight != 1:
            raise MocapyVMException("Sequence weighting is not supported")
        ess=self.ess_list[0]
        ess[0]+=self.weight*self.ess_r
        ess[1]+=self.weight*self.ess_w
        self.ess_r, self.ess_w=self._get_empty_ess()

    def sample(self, l):
        # Get which node value the parent node has in slice l
        parent_value, node_value=self.parentmap[l]

        # Sample from the corresponding von mises sampler
        sampler=self.samplers[parent_value]
        s=sampler()

        # Store the sampled value as the current node value

        self.parentmap[l]=s
        return s

    def blanket_sample(self,l):
        return self.sample(l)

    def get_slice_log_likelihood(self, l):
        # Get the value of the parent node in slice l
        # and the value of the node in slice l
        parent_value, node_value=self.parentmap[l]

        # Return log of the density at the node value
        # in the von mises distribution which corresponds to
        # the value of the parent node
        vm_dens=self.vm_list[parent_value]
        ll=vm_dens(node_value, log_space=1)
        return ll, (parent_value, node_value)

    def do_M_step(self, ess_list):
        # CPU 0
        if rank==0:
            mus=zeros((self.node_size,1), 'd')
            kappas=zeros(self.node_size, 'd')
            ess_r, ess_w=ess_list.pop()
            for i in range(0, len(ess_list)):
                r, w=ess_list[i]
                ess_r+=r
                ess_w+=w
            for i in range(0, self.node_size):
                r=ess_r[i]
                w=ess_w[i]
                norm_r=norm(r)

                print("do_M_step (%i): r=%s, w=%s, norm_r=%s" % (i,r,w,norm_r))

                if w == 0:
                    print("WARNING: There is no data to estimate mu and kappa from")
                    kappas[i]=self.kappas[i]
                    mus[i]=self.mus[i]
                else:
                    mu_coords=r/norm_r
                    kappas[i]=estimate_kappa(w,r,mu_coords)
                    mus[i]=to_radian(mu_coords)

            vm_list, samplers=self._make_vm_list(mus, kappas, self.node_size)
            self.mus, self.kappas, self.vm_list, self.samplers=\
                    bcast((mus, kappas, vm_list, samplers))
        # Other CPUs
        else:
            self.mus, self.kappas, self.vm_list, self.samplers=bcast()
        self.ess_list=[self._get_empty_ess()]

    def get_parameters(self):
        """
        Return the VM parameters.

        @return: Mu's, Kappa's
        @rtype: tuple of 2 numpy arrays
        """
        return self.mus, self.kappas

    def par2str(self, precision=2):
        s=""
        for i in range(0, self.node_size): 
            mu=self.mus[i]
            k=self.kappas[i]
            s=s+"Mu %i: %f\nKappa %i: %f\n\n" % (i, mu, i, k)
        return s[:-2]




In [30]:
i0_float = lambda x: float(i0(float(x)))

def to_radian(coords):
   if coords[1]>0:
      return acos(coords[0])
   else:
      return 2*pi-acos(coords[0])

def to_coords(angle):
   return array([cos(angle), sin(angle)])


def mod2pi(x):
   if x >= 2*pi:
      return mod2pi(x - 2*pi) 
   if x < 0:
      return mod2pi(x + 2*pi)
   else:
      return x


class VMSampler:
    """
    Sample from the Von Mises distribution.
    """
    def __init__(self, k, mu):
        """
        @param k: Kappa
        @type k: float

        @param mu: a 2-dimensional unit vector, VM mean direction
        @type mu: numpy array
        """
        # Make sure mu has the right shape
        if mu<0 or mu>2*pi:
            raise VMException("Mu has to lie in the interval between 0 and 2pi")
        else:
            # If so the parameters are stored (mu as radian)
           self.mu = mu
        self.k = k

    def __call__(self):
        """
        Generate a random vector sampled from the current Von Mises
        distribution.

        @return: random angle
        @rtype: float
        """

        try:
           value = vonmisesvariate(float(self.mu),float(self.k))
        except:
           print("ERROR: vonmisesvariate(%s,%s)" % (float(self.mu), float(self.k)))
           raise
        
        return mod2pi(value)


class VMDens:
    """
    VM density.
    """
    def __init__(self, k, mu):
        """
        @param k: Kappa
        @type k: float

        @param mu: an angle in the interval [0,2pi[, VM mean direction
        @type mu: float
        """
        
        # Make sure mu is in the correct interval
        if mu<0 or mu>2*pi:
            raise Exception("Mu has to lie between 0 and 2pi")
        
        #Store mu and kappa
        self.mu = mu
        self.k = k

        # Calculate and store the logarithm of the numerator in the
        # density function (= the reciprocal of the normalising constant)
        self.log_denom = log(2*pi*i0_float(k))



    def __call__(self, x, log_space=0):
        """
        Return VM density at x

        @rtype: float
        """
        
        # Calculate log of the numerator in the density function
        lognum = self.k*cos(x-self.mu)       

        # Calculate log of the density 
        logdens=lognum-self.log_denom
        
        if log_space:
            return logdens
        else:
            return exp(logdens)




In [31]:
MAX_KAPPA_VALUE = 500

def estimate_kappa(N, r, mu_coords):
    if N==0:
        print("WARNING: There is no data to estimate mu and kappa from")
        return MAX_KAPPA_VALUE+1

    else:    
        sum_cos = float(r[0])
        sum_sin = float(r[1])
        N = float(N)
        c = (1.0 / N) * sum_cos
        s = (1.0 / N) * sum_sin
        p = sqrt(pow(c,2) + pow(s,2))

        if(p < 2.0 / 3):
            return p*((2-pow(p,2)) / (1 - pow(p,2)))
        else:
            return (p + 1) / (4*p*(1-p))





In [32]:
#Gather data for a specific model on-the-fly
def coords_xyz_barn(coords):
    
    to_store = []
    for model in coords:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    info =atom.get_full_id()
                    coords = atom.get_coord()
                    to_store.append(coords)
                
    df = pd.DataFrame(np.row_stack(to_store))
    df.columns = ['x', 'y', 'z']
    return(df)

In [34]:
pathPDB = '/Users/eric1999/Desktop/Eric/'
pdbfile = "structure_%03d.pdb" % 999
#CODE WORKING
from random import randint

# The target sequence
seq = "GGCACUUCGGUGCC"

# Initialize the model
model = Barnacle(seq)

# Draw first sample
model.sample()
# Calculate and print likelihood
ll = model.get_log_likelihood()

model.save_structure_pdb("structure_%03d.pdb" % 999)
        
top = topology(pdbfile=pdbfile, path=pathPDB)

print("Structure one is finished yeah")

for i in range(50):
        # Choose position to sample
        j = randint(0, len(seq)-1)

        # Sample position j
        model.sample(start=j, end=j+1, num = i)

        # Calculate and print likelihood
        ll = model.get_log_likelihood()
        print("structure_%03i.pdb: ll = %f" % (i, ll))
        print("################################################")
        
        # Save the structure
        model.save_structure()

model.save_structure_pdb("structure_%03d_final.pdb")


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Previous energy
Finished generating the first sample ##############
Structure one is finished yeah
Previous energy
601.9923
Doing resampling
New energy
THERE IS < 1.5 DISTANCES
False
[601.9923, 601.645]
The random distribution is
[0.19637162]
structure_000.pdb: ll = 17.473303
################################################
Previous energy
601.645
Doing resampling
New energy
THERE IS < 1.5 DISTANCES
False
[601.645, 3268.4413]
The random distribution is
[0.97592492]
[601.645, 3268.4413]
Doing resampling
New energy
THERE IS < 1.5 DISTANCES
False
[601.645, 657.6817]
The random distribution is
[0.47496553]
[601.645, 657.6817]
Doing resampling
New energy
THERE IS < 1.5 DISTANCES
False
[601.645, 1245.9688]
The random distribution is
[0.21353669]
[601.645, 1245.9688]
Doing resampling
New energy
THERE IS < 1.5 DISTANCES
False
[601.645, 1081.6336]
The random distribution is
[0.6552001]
[601.645, 1081

KeyboardInterrupt: 